# Check time format and do imputation accordingly

In [3]:
import os
import pandas as pd
import numpy as np
lines = open("Hoppe_without.csv",'r').read().split('\n')
del lines[-1]
del lines[0]

# Data cleaning: find labor time, delivery date and BP information at certain time period for each patient

In [5]:
## Data processing to find specific patient information under certain ##
## conditions from specific dataset ##

MRN = []
delivery_time = []
bp_high = []
bp_low = []
labor_time = []

for line in lines:
    Lis = line.split(",")
    n = len(Lis)
    Lis[1]=pd.to_datetime(Lis[1]).strftime('%y-%m-%d %H:%M')
    for i in range(int((n-2)/3)):
        Lis[4+i*3]=pd.to_datetime(Lis[4+i*3]).strftime('%y-%m-%d %H:%M')
        if Lis[1]> Lis[4+i*3]:
            MRN.append(Lis[0])
            delivery_time.append(Lis[1])
            bp_high.append(Lis[2+i*3])
            bp_low.append(Lis[3+i*3])
            labor_time.append(Lis[4+i*3])
        
df = pd.DataFrame({'MRN': MRN,
                   'delivery_time': delivery_time,
                   'bp_high': bp_high,
                   'bp_low': bp_low,
                   'labor_time':labor_time
                  })

In [7]:
## Modify time format ##

a=df['delivery_time']
b=a.values.tolist()
delivery_date=[]
for x in b:
    #delivery_date.append(pd.to_datetime(x).strftime('%y-%m-%d'))
    delivery_date.append(pd.to_datetime(x, yearfirst = True).strftime('%y-%m-%d'))

In [8]:
df['delivery_date']=delivery_date

In [11]:
df_date = df.drop(['delivery_time'], axis=1)
df_date = df_date.drop_duplicates()

In [12]:
df_date['bp_high'] = [int(i) for i in df_date['bp_high']] 

In [13]:
## Find highest BP values ##

df_date.sort_values(['MRN','delivery_date','bp_high'],ascending=[1,1,0],inplace=True)
grouped = df_date.groupby(['MRN','delivery_date']).head(3)

In [ ]:
grouped1 = df_date.groupby(['MRN','delivery_date']).head(1)

In [17]:
grouped.columns = ["mother's_mrn", 'bp_high', 'bp_low', 'labor_time', 'date of_birth']

In [18]:
grouped1.columns = ["mother's_mrn", 'bp_high', 'bp_low', 'labor_time', 'date of_birth']

# Merge feature information from other datasets

In [21]:
HP = pd.ExcelFile('Hoppe_HTN_Data_9.2020.xlsx')
df1 = pd.read_excel(HP, 'Old Data')
df1['date of_birth']=df1['date of_birth'].apply(lambda x: x.strftime('%y-%m-%d'))

In [ ]:
pd.to_datetime(grouped['date of_birth'], yearfirst = True)

In [ ]:
df1['date of_birth'] = pd.to_datetime(df1['date of_birth'], yearfirst = True)
grouped['date of_birth'] = pd.to_datetime(grouped['date of_birth'], yearfirst = True)

In [ ]:
grouped1['date of_birth'] = pd.to_datetime(grouped1['date of_birth'], yearfirst = True)

In [ ]:
## Merge patient information on MRN and date of birth ##

grouped1["mother's_mrn"] = grouped1["mother's_mrn"].astype(str).astype(int)
olddff = pd.merge(df1, grouped1, on=["mother's_mrn",'date of_birth'])

In [29]:
## Data processing to find specific patient information under certain (after delivery 0-24h) ##
## conditions from specific dataset ##

import datetime
from dateutil.relativedelta import relativedelta
from datetime import date
MRN = []
delivery_time = []
bp_high24 = []
bp_low24 = []
labor_time = []


for line in lines:
    Lis = line.split(",")
    n = len(Lis)
    #list[1]=pd.to_datetime(list[1]).strftime('%y-%m-%d %H:%M')
    Lis[1]=pd.to_datetime(Lis[1])
    for i in range(int((n-2)/3)):
        #list[4+i*3]=pd.to_datetime(list[4+i*3]).strftime('%y-%m-%d %H:%M')
        Lis[4+i*3]=pd.to_datetime(Lis[4+i*3])
        diff = pd.Timedelta(Lis[4+i*3] - Lis[1], unit = 'hours').total_seconds()/3600
        if  0<diff<24:
            #print(list[4+i*3].strftime('%y-%m-%d %H:%M')+','+list[1].strftime('%y-%m-%d %H:%M'))
            MRN.append(Lis[0])
            delivery_time.append(Lis[1])
            bp_high24.append(Lis[2+i*3])
            bp_low24.append(Lis[3+i*3])
            labor_time.append(Lis[4+i*3])
        
df_1d = pd.DataFrame({'MRN': MRN,
                   'delivery_time': delivery_time,
                   'bp_high24': bp_high24,
                   'bp_low24': bp_low24,
                   'labor_time':labor_time
                  })

In [30]:
a=df_1d['delivery_time']
b=a.values.tolist()
delivery_date=[]
for x in b:
    #delivery_date.append(pd.to_datetime(x).strftime('%y-%m-%d'))
    delivery_date.append(pd.to_datetime(x, yearfirst = True).strftime('%y-%m-%d'))

In [31]:
df_1d['delivery_date']=delivery_date

In [32]:
df_date = df_1d.drop(['delivery_time'], axis=1)
df_date = df_date.drop_duplicates()

In [33]:
df_date['bp_high24'] = [int(i) for i in df_date['bp_high24']] 

In [34]:
df_date.sort_values(['MRN','delivery_date','bp_high24'],ascending=[1,1,0],inplace=True)
grouped = df_date.groupby(['MRN','delivery_date']).head(3)

In [ ]:
grouped1 = df_date.groupby(['MRN','delivery_date']).head(1)

In [37]:
grouped.columns = ["mother's_mrn", 'bp_high24', 'bp_low24', 'labor_time', 'date of_birth']
grouped1.columns = ["mother's_mrn", 'bp_high24', 'bp_low24', 'labor_time', 'date of_birth']

In [ ]:
grouped['date of_birth'] = pd.to_datetime(grouped['date of_birth'], yearfirst = True)
grouped1['date of_birth'] = pd.to_datetime(grouped1['date of_birth'], yearfirst = True)

In [ ]:
grouped1["mother's_mrn"] = grouped1["mother's_mrn"].astype(str).astype(int)
olddfff = pd.merge(olddff, grouped1, on=["mother's_mrn",'date of_birth'])

In [ ]:
## Data processing to find specific patient information under certain (after delivery 24-48h) ##
## conditions from specific dataset ##

import datetime
from dateutil.relativedelta import relativedelta
from datetime import date
MRN = []
delivery_time = []
bp_high48 = []
bp_low48 = []
labor_time = []


for line in lines:
    Lis = line.split(",")
    n = len(Lis)
    #list[1]=pd.to_datetime(list[1]).strftime('%y-%m-%d %H:%M')
    Lis[1]=pd.to_datetime(Lis[1])
    for i in range(int((n-2)/3)):
        #list[4+i*3]=pd.to_datetime(list[4+i*3]).strftime('%y-%m-%d %H:%M')
        Lis[4+i*3]=pd.to_datetime(Lis[4+i*3])
        diff = pd.Timedelta(Lis[4+i*3] - Lis[1], unit = 'hours').total_seconds()/3600
        if  24<diff<48:
            print(Lis[4+i*3].strftime('%y-%m-%d %H:%M')+','+Lis[1].strftime('%y-%m-%d %H:%M'))
            MRN.append(Lis[0])
            delivery_time.append(Lis[1])
            bp_high48.append(Lis[2+i*3])
            bp_low48.append(Lis[3+i*3])
            labor_time.append(Lis[4+i*3])
        
df_2d = pd.DataFrame({'MRN': MRN,
                   'delivery_time': delivery_time,
                   'bp_high48': bp_high48,
                   'bp_low48': bp_low48,
                   'labor_time':labor_time
                  })

In [42]:
a=df_2d['delivery_time']
b=a.values.tolist()
delivery_date=[]
for x in b:
    #delivery_date.append(pd.to_datetime(x).strftime('%y-%m-%d'))
    delivery_date.append(pd.to_datetime(x, yearfirst = True).strftime('%y-%m-%d'))

In [43]:
df_2d['delivery_date']=delivery_date

In [44]:
df_date = df_2d.drop(['delivery_time'], axis=1)
df_date = df_date.drop_duplicates()

In [45]:
df_date['bp_high48'] = [int(i) for i in df_date['bp_high48']] 

In [46]:
df_date.sort_values(['MRN','delivery_date','bp_high48'],ascending=[1,1,0],inplace=True)
grouped = df_date.groupby(['MRN','delivery_date']).head(3)

In [47]:
grouped1 = df_date.groupby(['MRN','delivery_date']).head(1)

In [48]:
grouped.columns = ["mother's_mrn", 'bp_high48', 'bp_low48', 'labor_time', 'date of_birth']
grouped1.columns = ["mother's_mrn", 'bp_high48', 'bp_low48', 'labor_time', 'date of_birth']

In [ ]:
grouped['date of_birth'] = pd.to_datetime(grouped['date of_birth'], yearfirst = True)
grouped1['date of_birth'] = pd.to_datetime(grouped1['date of_birth'], yearfirst = True)

In [ ]:
grouped1["mother's_mrn"] = grouped1["mother's_mrn"].astype(str).astype(int)
olddffff = pd.merge(olddfff, grouped1, on=["mother's_mrn",'date of_birth'])

# Demographic infomation collection

In [ ]:
olddemo = olddffff[["mother's_mrn",'maternal_age','bp_high24','bp_low24','bp_high48','bp_low48','bp_high','bp_low',"m9's bmi_prepregnancy",'nullipara/_multipara',"gestational_age (ob_clinician's_final estimate-_calculated)",'primary_race','final_route and_method of_delivery','hispanic_origin','READMIT_ ADM_DATE_TIME_1','READMIT_ ADM_DATE_TIME_2']]
olddemo.head()

In [ ]:
import numpy as np
#throw NaN in each column.
olddemo = olddemo.replace('.', np.nan)
olddemo.dropna(subset=['bp_high24','bp_low24','bp_high48','bp_low48', 'bp_high', 'bp_low', "m9's bmi_prepregnancy"], how='any', axis=0, inplace = True)
print(olddemo.shape)
olddemo.head()
olddemo.info()

In [606]:
olddemo['READMIT_ ADM_DATE_TIME_1'].fillna(0,inplace=True)
olddemo['READMIT_ ADM_DATE_TIME_1']=olddemo['READMIT_ ADM_DATE_TIME_1'].apply(lambda x: 1 if x!=0 else 0)
olddemo['READMIT_ ADM_DATE_TIME_2'].fillna(0,inplace=True)
olddemo['READMIT_ ADM_DATE_TIME_2']=olddemo['READMIT_ ADM_DATE_TIME_2'].apply(lambda x: 1 if x!=0 else 0)

In [ ]:
readmit = np.logical_or(olddemo['READMIT_ ADM_DATE_TIME_1'] > 0, olddemo['READMIT_ ADM_DATE_TIME_2'] > 0)
olddemo['Readmit'] = readmit
# change True False to 1 0
olddemo['Readmit'] = olddemo['Readmit']*1
olddemo.head()

In [279]:
oldbpdiag['bp_low'] = oldbpdiag['bp_low'].astype(int)
oldbpdiag['bp_low24'] = oldbpdiag['bp_low24'].astype(int)
oldbpdiag['bp_low48'] = oldbpdiag['bp_low48'].astype(int)

In [407]:
oldps = oldbpdiag["mother's_mrn"]

In [281]:
## Label patients under certain criteria ##

Index_label1 = oldbpdiag[(oldbpdiag['bp_low']>90)|(oldbpdiag['bp_high']>140)].index.tolist() 

In [282]:
Index_label2 = oldbpdiag[(oldbpdiag['bp_low24']>90)|(oldbpdiag['bp_high24']>140)].index.tolist() 

In [283]:
Index_label3 = oldbpdiag[(oldbpdiag['bp_low48']>90)|(oldbpdiag['bp_high48']>140)].index.tolist() 

In [284]:
qulify1_set = set.intersection(set(Index_label1), set(Index_label2))
qulify2_set = set.intersection(set(Index_label1), set(Index_label3))
qulify3_set = set.intersection(set(Index_label2), set(Index_label3))

In [285]:
oldqulify = qulify1_set.union(qulify2_set, qulify3_set)

In [288]:
Index_label4 = oldbpdiag[(oldbpdiag['protein_24_hour_ur_fst_tm'].isnull())].index.tolist()

In [289]:
Index_label5 = oldbpdiag[(oldbpdiag['protein_24_hour_ur_fst_tm'].notnull())].index.tolist()

In [290]:
gesqualify = set.intersection(set(oldqulify), set(Index_label4))

In [291]:
preeclamquali = set.intersection(set(oldqulify), set(Index_label5))

In [ ]:
olddffff_key = olddffff[['READMIT_ ADM_DATE_TIME_1','READMIT_ ADM_DATE_TIME_2','bp_high24','bp_low24','bp_high48','bp_low48',"m9's bmi_prepregnancy","gestational_age (ob_clinician's_final estimate-_calculated)",'maternal_age', 'bp_high', 'bp_low']]
summary_key=olddffff_key.isnull().sum()
summary_key

# Old data without med information

In [ ]:
import numpy as np
#throw NaN in each column.
olddffff_key = olddffff_key.replace('.', np.nan)
olddffff_key.dropna(subset=['bp_high24','bp_low24','bp_high48','bp_low48',"m9's bmi_prepregnancy","gestational_age (ob_clinician's_final estimate-_calculated)",'maternal_age', 'bp_high', 'bp_low'], how='any', axis=0, inplace = True)
print(olddffff_key.shape)
olddffff_key.head()
olddffff_key.info()

In [57]:
olddffff_key['READMIT_ ADM_DATE_TIME_1'].fillna(0,inplace=True)
olddffff_key['READMIT_ ADM_DATE_TIME_1']=olddffff_key['READMIT_ ADM_DATE_TIME_1'].apply(lambda x: 1 if x!=0 else 0)
olddffff_key['READMIT_ ADM_DATE_TIME_2'].fillna(0,inplace=True)
olddffff_key['READMIT_ ADM_DATE_TIME_2']=olddffff_key['READMIT_ ADM_DATE_TIME_2'].apply(lambda x: 1 if x!=0 else 0)

In [ ]:
readmit = np.logical_or(olddffff_key['READMIT_ ADM_DATE_TIME_1'] > 0, olddffff_key['READMIT_ ADM_DATE_TIME_2'] > 0)
olddffff_key['Readmit'] = readmit
# change True False to 1 0
olddffff_key['Readmit'] = olddffff_key['Readmit']*1
olddffff_key.head()

# Medication data cleaning

In [ ]:
#Med data
MD = pd.ExcelFile('Old Data - Only MRN and Medication.xlsx')
Medinfo = pd.read_excel(MD, 'Sheet1')
print(Medinfo.shape)

In [ ]:
Mednodup = Medinfo.drop_duplicates()
Mednodup.head()

In [63]:
Medlist = Mednodup.values.tolist()

In [67]:
## Data processing to find specific medication information under certain ##
## conditions from specific dataset ##

MRN = []
med_time = []
med_name = []
med_dose = []
dose_unit = []
route = []

n=131

for j in range(len(Mednodup)):
    for i in range(int((n-1)/5)):
        MRN.append(Mednodup.iloc[j,0])
        med_time.append(Mednodup.iloc[j,5*i+1])
        med_name.append(Mednodup.iloc[j,5*i+2])
        med_dose.append(Mednodup.iloc[j,5*i+3])
        dose_unit.append(Mednodup.iloc[j,5*i+4])
        route.append(Mednodup.iloc[j,5*i+5])
        
df_minfo = pd.DataFrame({'MRN': MRN,
                   'med_time': med_time,
                   'med_name': med_name,
                   'med_dose': med_dose,
                   'dose_unit': dose_unit,
                   'route':route
                  })

# Parsing medication information from text format

In [68]:
df_minfo = df_minfo.replace('15-30', 30)
df_minfo = df_minfo.replace('400-800', 400)

In [69]:
df_minfo.loc[df_minfo['med_dose'].apply(lambda x: isinstance(x,datetime.date)),'med_dose'] = 20

In [ ]:
df_minfo.loc[df_minfo['MRN'] == 409754]

In [71]:
M1 = (
    df_minfo.groupby('MRN')
    .apply(lambda x: (x['med_name'].eq(1)).any())
    .to_frame('If Ibuprofen'))

In [72]:
df_minfo["ifnacute"] = ((df_minfo["med_name"] == 2)&(df_minfo["med_dose"].fillna(-1)>=10)&(df_minfo["med_dose"].fillna(-1)<=20))

In [ ]:
df_minfo[(df_minfo["med_name"] == 2)&(df_minfo["med_dose"].fillna(-1)>=10)&(df_minfo["med_dose"].fillna(-1)<=20)]

In [74]:
df_minfo["ifnsustain"] = ((df_minfo["med_name"] == 2)&(df_minfo["med_dose"].fillna(-1)>=30)&(df_minfo["med_dose"].fillna(-1)<=90))

In [75]:
M2_1 = (
    df_minfo.groupby('MRN')
    .apply(lambda x: (x['ifnacute'].eq(True)).any())
    .to_frame('If nifedipine acute'))

In [ ]:
M2_1.head(5)

In [77]:
M2_2 = (
    df_minfo.groupby('MRN')
    .apply(lambda x: (x['ifnsustain'].eq(True)).any())
    .to_frame('If nifedipine sustain'))

In [ ]:
df_minfo[(df_minfo["med_name"] == 2)&(df_minfo["med_dose"].fillna(-1)>=30)&(df_minfo["med_dose"].fillna(-1)<=90)]

In [ ]:
M2_2.head(5)

In [80]:
M3_1 = (
    df_minfo.groupby('MRN')
    .apply(lambda x: (x['med_name'].eq(3) & x['route'].eq('IV')).any())
    .to_frame('If Labetalol given IV'))

In [81]:
M3_2 = (
    df_minfo.groupby('MRN')
    .apply(lambda x: (x['med_name'].eq(3) & x['route'].eq('PO')).any())
    .to_frame('If Labetalol given PO'))

In [82]:
M6 = (
    df_minfo.groupby('MRN')
    .apply(lambda x: (x['med_name'].eq(6)).any())
    .to_frame('If hydralazine'))

In [ ]:
M6.head(5)

In [ ]:
Mdata = pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(M1,M2_1,on='MRN'),M2_2,on='MRN'),M3_1,on='MRN'),M3_2,on='MRN'),M6,on='MRN')
Mdata.head()

In [85]:
Mdata1 = Mdata.reset_index()

In [87]:
Mdata1.columns = ["mother's_mrn", 'If Ibuprofen', 'If nifedipine acute', 'If nifedipine sustain', 'If Labetalol given IV', 'If Labetalol given PO', 'If hydralazine']
olddfwithmed = pd.merge(olddffff, Mdata1, on=["mother's_mrn"])

# Old data with medication information

In [ ]:
olddfwithmed_key = olddfwithmed[['READMIT_ ADM_DATE_TIME_1','READMIT_ ADM_DATE_TIME_2','bp_high24','bp_low24','bp_high48','bp_low48',"m9's bmi_prepregnancy","gestational_age (ob_clinician's_final estimate-_calculated)",'maternal_age', 'bp_high', 'bp_low','If Ibuprofen', 'If nifedipine acute', 'If nifedipine sustain', 'If Labetalol given IV', 'If Labetalol given PO', 'If hydralazine']]
summary_key=olddfwithmed_key.isnull().sum()
summary_key

In [ ]:
olddfwithmed_key = olddfwithmed_key.replace('.', np.nan)
olddfwithmed_key.dropna(subset=['bp_high24','bp_low24','bp_high48','bp_low48',"m9's bmi_prepregnancy","gestational_age (ob_clinician's_final estimate-_calculated)",'maternal_age', 'bp_high', 'bp_low','If Ibuprofen', 'If nifedipine acute', 'If nifedipine sustain', 'If Labetalol given IV', 'If Labetalol given PO', 'If hydralazine'], how='any', axis=0, inplace = True)
print(olddfwithmed_key.shape)
olddfwithmed_key.head()
olddfwithmed_key.info()

In [ ]:
olddfwithmed_key['READMIT_ ADM_DATE_TIME_1'].fillna(0,inplace=True)
olddfwithmed_key['READMIT_ ADM_DATE_TIME_1']=olddfwithmed_key['READMIT_ ADM_DATE_TIME_1'].apply(lambda x: 1 if x!=0 else 0)
olddfwithmed_key['READMIT_ ADM_DATE_TIME_2'].fillna(0,inplace=True)
olddfwithmed_key['READMIT_ ADM_DATE_TIME_2']=olddfwithmed_key['READMIT_ ADM_DATE_TIME_2'].apply(lambda x: 1 if x!=0 else 0)
# change True False to 1 0
olddfwithmed_key['If Ibuprofen'] = olddfwithmed_key['If Ibuprofen']*1
olddfwithmed_key['If nifedipine acute'] = olddfwithmed_key['If nifedipine acute']*1
olddfwithmed_key['If nifedipine sustain'] = olddfwithmed_key['If nifedipine sustain']*1
olddfwithmed_key['If Labetalol given IV'] = olddfwithmed_key['If Labetalol given IV']*1
olddfwithmed_key['If Labetalol given PO'] = olddfwithmed_key['If Labetalol given PO']*1
olddfwithmed_key['If hydralazine'] = olddfwithmed_key['If hydralazine']*1

In [ ]:
readmit = np.logical_or(olddfwithmed_key['READMIT_ ADM_DATE_TIME_1'] > 0, olddfwithmed_key['READMIT_ ADM_DATE_TIME_2'] > 0)
olddfwithmed_key['Readmit'] = readmit
# change True False to 1 0
olddfwithmed_key['Readmit'] = olddfwithmed_key['Readmit']*1
olddfwithmed_key.head()

In [ ]:
olddfwithmed_key.loc[olddfwithmed_key['Readmit']==1]

In [94]:
olddfwithmed_key.to_csv('olddata.csv')

# Same thing for the new data

In [96]:
HP = pd.ExcelFile('Hoppe_HTN_Data_9.2020.xlsx')
dfnew = pd.read_excel(HP, 'New Data')

In [97]:
dfnew_laborbp = dfnew[["mother's_mrn",'sysbp_labor1','diabp_labor1','sysbp_labor2','diabp_labor2','sysbp_labor3','diabp_labor3']]

In [99]:
MRN = []
bp_high = []
bp_low = []

n=7

for j in range(len(dfnew_laborbp)):
    for i in range(int((n-1)/2)):
        MRN.append(dfnew_laborbp.iloc[j,0])
        bp_high.append(dfnew_laborbp.iloc[j,2*i+1])
        bp_low.append(dfnew_laborbp.iloc[j,2*i+2])
        
dfnew_bpns = pd.DataFrame({'MRN': MRN,
                   'bp_high': bp_high,
                   'bp_low': bp_low
                  })

In [101]:
dfnew_bpns = dfnew_bpns.drop_duplicates()

In [102]:
dfnew_bpns = dfnew_bpns.dropna()

In [103]:
dfnew_bpns['bp_high'] = [float(i) for i in dfnew_bpns['bp_high']] 
dfnew_bpns.sort_values(['MRN','bp_high'],ascending=[1,0],inplace=True)
grouped_1 = dfnew_bpns.groupby(['MRN']).head(1)

In [108]:
grouped_1.columns = ["mother's_mrn", 'bp_high', 'bp_low']

In [ ]:
newdfbp = pd.merge(grouped_1,dfnew,on="mother's_mrn")
newdfbp.head()

In [ ]:
newdfbp = newdfbp.drop_duplicates(subset=["mother's_mrn", 'bp_high', 'bp_low'], keep='last')
newdfbp.head()

# New data Med info

In [ ]:
dfnew_med = pd.read_excel('January_2020_-_15_through_18_data_combinedv4.xlsx', 'Meds 15-19')

In [108]:
dfnew_med = dfnew_med.drop_duplicates()

In [110]:
dfnew_med['If Ibuprofen'] = dfnew_med['Medication Name'].str.contains("ibuprofen")

In [112]:
dfnew_med['If nifedipine sustain'] = (dfnew_med['Medication Name'].str.contains("NIFEdipine")) & (dfnew_med['Medication Name'].str.contains("tablet"))
dfnew_med['If nifedipine acute'] = (dfnew_med['Medication Name'].str.contains("NIFEdipine")) & (dfnew_med['Medication Name'].str.contains("capsule"))

In [114]:
dfnew_med['If Labetalol given IV'] = (dfnew_med['Medication Name'].str.contains("labetalol")) & (dfnew_med['Medication Name'].str.contains("injection|syringe|infusion"))
dfnew_med['If Labetalol given PO'] = (dfnew_med['Medication Name'].str.contains("labetalol")) & (dfnew_med['Medication Name'].str.contains("tablet"))

In [116]:
dfnew_med['If hydralazine'] = dfnew_med['Medication Name'].str.contains("hydrALAZINE")

In [118]:
bool_cols = dfnew_med.columns[dfnew_med.dtypes == 'bool']
dfnew_med[bool_cols] = dfnew_med[bool_cols].replace({True: 'Yes', False: 'No'})

In [120]:
NM_1 = (
    dfnew_med.groupby('id')
    .apply(lambda x: x['If Ibuprofen'].eq('Yes').any())
    .to_frame('If Ibuprofen'))

In [121]:
NM_21 = (
    dfnew_med.groupby('id')
    .apply(lambda x: x['If nifedipine acute'].eq('Yes').any())
    .to_frame('If nifedipine acute'))

In [122]:
NM_22 = (
    dfnew_med.groupby('id')
    .apply(lambda x: x['If nifedipine sustain'].eq('Yes').any())
    .to_frame('If nifedipine sustain'))

In [123]:
NM_31 = (
    dfnew_med.groupby('id')
    .apply(lambda x: x['If Labetalol given IV'].eq('Yes').any())
    .to_frame('If Labetalol given IV'))

In [124]:
NM_32 = (
    dfnew_med.groupby('id')
    .apply(lambda x: x['If Labetalol given PO'].eq('Yes').any())
    .to_frame('If Labetalol given PO'))

In [125]:
NM_6 = (
    dfnew_med.groupby('id')
    .apply(lambda x: x['If hydralazine'].eq('Yes').any())
    .to_frame('If hydralazine'))

In [ ]:
NMdata = pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(NM_1,NM_21,on='id'),NM_22,on='id'),NM_31,on='id'),NM_32,on='id'),NM_6,on='id')
NMdata.head()

In [ ]:
NMdata1 = NMdata.reset_index()
NMdata1.head()

In [129]:
NMdata1.columns = ["mother's_mrn", 'If Ibuprofen', 'If nifedipine acute', 'If nifedipine sustain', 'If Labetalol given IV', 'If Labetalol given PO', 'If hydralazine']
newdfwithmed = newdfbp.merge(NMdata1, on="mother's_mrn", how='left')

In [131]:
newdfwithmed[['If Ibuprofen', 'If nifedipine acute', 'If nifedipine sustain', 'If Labetalol given IV', 'If Labetalol given PO', 'If hydralazine']] = newdfwithmed[['If Ibuprofen', 'If nifedipine acute', 'If nifedipine sustain', 'If Labetalol given IV', 'If Labetalol given PO', 'If hydralazine']].fillna(False).astype(bool)

# Demographics for new data and combined data

In [133]:
newdemo = newdfwithmed[["mother's_mrn",'maternal_age','sysbp_24deliv1','diabp_24deliv1','sysbp_24_48deliv1','diabp_24_48deliv1','bp_high','bp_low',"mother's bmi_prepregnancy",'nullipara/_multipara',"gestational_age (ob_clinician's_final estimate-_calculated)",'primary_race','hispanic_origin','final_route and_method of_delivery','READMIT_ ADM_DATE_TIME_1','READMIT_ ADM_DATE_TIME_2']]


In [ ]:
newdemo.dropna(subset=['sysbp_24deliv1','diabp_24deliv1','sysbp_24_48deliv1','diabp_24_48deliv1',"mother's bmi_prepregnancy","gestational_age (ob_clinician's_final estimate-_calculated)",'maternal_age', 'bp_high', 'bp_low'], how='any', axis=0, inplace = True)
print(newdemo.shape)
newdemo.head()
newdemo.info()

In [ ]:
newdemo['READMIT_ ADM_DATE_TIME_1'].fillna(0,inplace=True)
newdemo['READMIT_ ADM_DATE_TIME_1']=newdemo['READMIT_ ADM_DATE_TIME_1'].apply(lambda x: 1 if x!=0 else 0)
newdemo['READMIT_ ADM_DATE_TIME_2'].fillna(0,inplace=True)
newdemo['READMIT_ ADM_DATE_TIME_2']=newdemo['READMIT_ ADM_DATE_TIME_2'].apply(lambda x: 1 if x!=0 else 0)

In [ ]:
newreadmit = np.logical_or(newdemo['READMIT_ ADM_DATE_TIME_1'] > 0, newdemo['READMIT_ ADM_DATE_TIME_2'] > 0)
newdemo['Readmit'] = newreadmit
# change True False to 1 0
newdemo['Readmit'] = newdemo['Readmit']*1
newdemo.head()

In [615]:
newdemo = newdemo[["mother's_mrn", "maternal_age", "sysbp_24deliv1", "diabp_24deliv1", "sysbp_24_48deliv1", "diabp_24_48deliv1", "bp_high", "bp_low", "mother's bmi_prepregnancy", "nullipara/_multipara", "gestational_age (ob_clinician's_final estimate-_calculated)", "primary_race", "final_route and_method of_delivery", "hispanic_origin", "READMIT_ ADM_DATE_TIME_1", "READMIT_ ADM_DATE_TIME_2", "Readmit"]]

In [619]:
newdemo=newdemo.rename(columns={"sysbp_24deliv1": "bp_high24", "diabp_24deliv1": "bp_low24", "sysbp_24_48deliv1": "bp_high48", "diabp_24_48deliv1": "bp_low48", "mother's bmi_prepregnancy": "m9's bmi_prepregnancy"})

In [621]:
alldemo=olddemo.append(newdemo)

In [624]:
alldemo['nullipara/_multipara']=alldemo['nullipara/_multipara'].fillna(2)
alldemo['primary_race']=alldemo['primary_race'].fillna(0)

In [ ]:
alldemo.groupby(['Readmit'])['maternal_age'].mean()

In [ ]:
alldemo.groupby(['Readmit'])['maternal_age'].std()

In [ ]:
alldemo.groupby(['Readmit'])['nullipara/_multipara'].value_counts()

In [ ]:
alldemo.groupby(['Readmit'])["gestational_age (ob_clinician's_final estimate-_calculated)"].mean()

In [ ]:
alldemo.groupby(['Readmit'])["gestational_age (ob_clinician's_final estimate-_calculated)"].std()

In [ ]:
alldemo.groupby(['Readmit'])['primary_race'].value_counts()

In [ ]:
alldemo.groupby(['Readmit'])['hispanic_origin'].value_counts()

In [ ]:
alldemo.groupby(['Readmit'])["m9's bmi_prepregnancy"].mean()

In [ ]:
alldemo.groupby(['Readmit'])["m9's bmi_prepregnancy"].std()

In [ ]:
alldemo.groupby(['Readmit'])['final_route and_method of_delivery'].value_counts()

In [ ]:
alldemo['maternal_age'].mean()

In [ ]:
alldemo['maternal_age'].std()

In [ ]:
alldemo["gestational_age (ob_clinician's_final estimate-_calculated)"].mean()

In [ ]:
alldemo["gestational_age (ob_clinician's_final estimate-_calculated)"].std()

In [ ]:
alldemo["m9's bmi_prepregnancy"].mean()

In [ ]:
alldemo["m9's bmi_prepregnancy"].std()

# Formatting dataframe for new data information

In [ ]:
newdfwithmed_key = newdfwithmed[['READMIT_ ADM_DATE_TIME_1','READMIT_ ADM_DATE_TIME_2','sysbp_24deliv1','diabp_24deliv1','sysbp_24_48deliv1','diabp_24_48deliv1',"mother's bmi_prepregnancy","gestational_age (ob_clinician's_final estimate-_calculated)",'maternal_age', 'bp_high', 'bp_low','If Ibuprofen', 'If nifedipine acute', 'If nifedipine sustain', 'If Labetalol given IV', 'If Labetalol given PO', 'If hydralazine']]
summary_key=newdfwithmed_key.isnull().sum()
summary_key

In [ ]:
newdfwithmed_key.dropna(subset=['sysbp_24deliv1','diabp_24deliv1','sysbp_24_48deliv1','diabp_24_48deliv1',"mother's bmi_prepregnancy","gestational_age (ob_clinician's_final estimate-_calculated)",'maternal_age', 'bp_high', 'bp_low','If Ibuprofen', 'If nifedipine acute', 'If nifedipine sustain', 'If Labetalol given IV', 'If Labetalol given PO', 'If hydralazine'], how='any', axis=0, inplace = True)
print(newdfwithmed_key.shape)
newdfwithmed_key.head()
newdfwithmed_key.info()

In [ ]:
newps = newdfwithmed_key["mother's_mrn"]

In [ ]:
newdfwithmed_key['READMIT_ ADM_DATE_TIME_1'].fillna(0,inplace=True)
newdfwithmed_key['READMIT_ ADM_DATE_TIME_1']=newdfwithmed_key['READMIT_ ADM_DATE_TIME_1'].apply(lambda x: 1 if x!=0 else 0)
newdfwithmed_key['READMIT_ ADM_DATE_TIME_2'].fillna(0,inplace=True)
newdfwithmed_key['READMIT_ ADM_DATE_TIME_2']=newdfwithmed_key['READMIT_ ADM_DATE_TIME_2'].apply(lambda x: 1 if x!=0 else 0)
# change True False to 1 0
newdfwithmed_key['If Ibuprofen'] = newdfwithmed_key['If Ibuprofen']*1
newdfwithmed_key['If nifedipine acute'] = newdfwithmed_key['If nifedipine acute']*1
newdfwithmed_key['If nifedipine sustain'] = newdfwithmed_key['If nifedipine sustain']*1
newdfwithmed_key['If Labetalol given IV'] = newdfwithmed_key['If Labetalol given IV']*1
newdfwithmed_key['If Labetalol given PO'] = newdfwithmed_key['If Labetalol given PO']*1
newdfwithmed_key['If hydralazine'] = newdfwithmed_key['If hydralazine']*1

In [ ]:
newreadmit = np.logical_or(newdfwithmed_key['READMIT_ ADM_DATE_TIME_1'] > 0, newdfwithmed_key['READMIT_ ADM_DATE_TIME_2'] > 0)
newdfwithmed_key['Readmit'] = newreadmit
# change True False to 1 0
newdfwithmed_key['Readmit'] = newdfwithmed_key['Readmit']*1
newdfwithmed_key.head()

In [141]:
newdfwithmed_key.columns = ['READMIT_ ADM_DATE_TIME_1','READMIT_ ADM_DATE_TIME_2','bp_high24','bp_low24','bp_high48','bp_low48',"m9's bmi_prepregnancy","gestational_age (ob_clinician's_final estimate-_calculated)",'maternal_age','bp_high','bp_low','If Ibuprofen','If nifedipine acute','If nifedipine sustain','If Labetalol given IV','If Labetalol given PO','If hydralazine','Readmit']

In [142]:
newdfwithmed_key = newdfwithmed_key[['READMIT_ ADM_DATE_TIME_1','READMIT_ ADM_DATE_TIME_2','bp_high24','bp_low24','bp_high48','bp_low48',"m9's bmi_prepregnancy","gestational_age (ob_clinician's_final estimate-_calculated)",'maternal_age','bp_high','bp_low','If Ibuprofen','If nifedipine acute','If nifedipine sustain','If Labetalol given IV','If Labetalol given PO','If hydralazine','Readmit']]

In [ ]:
newdfwithmed_key.loc[newdfwithmed_key['Readmit']==1]

# Combined data

In [145]:
combineddf = olddfwithmed_key.append(newdfwithmed_key)

In [4]:
combineddf.to_csv('mergeddata.csv')

In [3]:
import pandas as pd 
combineddf = pd.read_csv('mergeddata.csv') 

In [5]:
X = combineddf.drop(['Unnamed: 0','READMIT_ ADM_DATE_TIME_1','READMIT_ ADM_DATE_TIME_2','Readmit'],axis=1)
y = combineddf.Readmit

In [ ]:
## Check feature correlation ##

X.rename(columns={"m9's bmi_prepregnancy": "bmi_prepregnancy", "gestational_age (ob_clinician's_final estimate-_calculated)": "gestational_age"}, inplace = True)
corr = X.corr()

import seaborn as sns
ax = sns.heatmap(
    corr, 
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
);
plt.savefig("all_data_corr.png", dpi=300)

In [7]:
#old and new data sets
X_old, y_old = X.iloc[0:21037], y.iloc[0:21037]
X_new, y_new = X.iloc[21037:], y.iloc[21037:]

In [ ]:
corr_old = X_old.corr()

In [ ]:
corr_new = X_new.corr()

In [ ]:
## Check feature correlation ##

X_old.rename(columns={"m9's bmi_prepregnancy": "bmi_prepregnancy", "gestational_age (ob_clinician's_final estimate-_calculated)": "gestational_age"}, inplace = True)
corr = X_old.corr()

ax = sns.heatmap(
    corr, 
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
);
plt.savefig("old_data_corr.png", dpi=300)

In [ ]:
## Check feature correlation ##

X_new.rename(columns={"m9's bmi_prepregnancy": "bmi_prepregnancy", "gestational_age (ob_clinician's_final estimate-_calculated)": "gestational_age"}, inplace = True)
corr = X_new.corr()

ax = sns.heatmap(
    corr, 
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
);

In [8]:
X_new = X_new.reset_index(drop=True)

In [3]:
## If no BP features? ##
X_nobp = combineddf.drop(['Unnamed: 0','bp_high24','bp_low24','bp_high48','bp_low48','bp_high','bp_low','READMIT_ ADM_DATE_TIME_1','READMIT_ ADM_DATE_TIME_2','Readmit'],axis=1) 

# Machine learning process

In [154]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_o = scaler.fit_transform(X_old)
scaled_n = scaler.fit_transform(X_new)

In [9]:
y_new = y_new.reset_index(drop=True)

In [10]:
from sklearn.model_selection import StratifiedKFold

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score, auc
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
## StratifiedKFold cross validation hyperparameter tuning ##

balances = [{0: 1, 1: 1}, {0: 1, 1: 200}, {0: 1, 1: 300}, {0: 1, 1: 500}, {0: 1, 1: 1000}]
m_dep = [2, 4, 6, 8, 10, 50, 100]
n_est = [5, 10, 100, 500, 1000]
#m_fea = ['sqrt','log2']

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=10)
#main loop for tuning
rf_o_result_dfs = []
for balance in balances:
    print('I am bal')
    lis = []
    for i in m_dep:
        for j in n_est:
            lst_bala_accu_stratified = []
            lst_spe_stratified = []
            lst_sens_stratified = []
            lst_exp_cost = []
            lst_pre = []
            lst_npv = []
            for train_index, test_index in skf.split(scaled_o, y_old):
                X_train_fold, X_test_fold = scaled_o[train_index], scaled_o[test_index]
                y_train_fold, y_test_fold = y_old[train_index], y_old[test_index]
                RF_co = RandomForestClassifier(max_depth = i, n_estimators = j, class_weight=balance)
                RF_co.fit(X_train_fold, y_train_fold)
                RF_co_prd=RF_co.predict(X_test_fold)
                tn, fp, fn, tp = confusion_matrix(y_test_fold, RF_co_prd).ravel()
                ba = 1./2*(1.*tp/(tp+fn)+1.*tn/(tn+fp))
                spe = 1.*tn/(tn+fp)
                sens = 1.*tp/(tp+fn)
                pres = tp/(tp+fp)
                npvs = tn/(tn+fn)
                cos = 565.*fn + 1.*fp
                lst_bala_accu_stratified.append(ba)
                lst_spe_stratified.append(spe)
                lst_sens_stratified.append(sens)
                lst_exp_cost.append(cos)
                lst_pre.append(pres)
                lst_npv.append(npvs)
            #print(np.mean(lst_bala_accu_stratified))
            lis.append([balance,i,j,np.mean(lst_spe_stratified),np.mean(lst_sens_stratified),np.mean(lst_bala_accu_stratified),
                        np.mean(lst_pre),np.mean(lst_npv),np.mean(lst_exp_cost)])
    lis_dfOps = pd.DataFrame(lis)
    df_rf_baOps = lis_dfOps.sort_values(by=[5], ascending=False)
    rf_o_result_dfs.append(df_rf_baOps.head(1))
final_Oresult_df = pd.concat(rf_o_result_dfs, ignore_index=True)
#df_rf_pc5p = lis_df5p.sort_values(by=[5], ascending=False)
#print(df_rf_pc5p.head(1))

In [216]:
## Calculate F1 and medical cost ##

final_Oresult_df['f1'] = 2* final_Oresult_df[4] * final_Oresult_df[6]/(final_Oresult_df[4] + final_Oresult_df[6])
final_Oresult_df['cost'] = final_Oresult_df[8] * 36

In [ ]:
## Get feature importance ##


from sklearn.model_selection import cross_val_score, KFold
balance = {0: 1, 1: 200}
n_folds = 5
kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

rf_classifier = RandomForestClassifier(max_depth = 2, n_estimators = 1000, class_weight=balance)
feature_importances_list = []
for train_index, test_index in kf.split(X_old):
    X_train, X_test = X_old.loc[train_index], X_old.loc[test_index]
    y_train, y_test = y_old[train_index], y_old[test_index]

    rf_classifier.fit(X_train, y_train)
    feature_importances_list.append(rf_classifier.feature_importances_)
average_feature_importances = np.mean(feature_importances_list, axis=0)
feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': average_feature_importances})


feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

print(feature_importance_df)

In [ ]:
balance = {0: 1, 1: 200}
n_folds = 5
kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

rf_classifier = RandomForestClassifier(max_depth = 2, n_estimators = 1000, class_weight=balance)
feature_importances_list = []
for train_index, test_index in kf.split(X_new):
    X_train, X_test = X_new.loc[train_index], X_new.loc[test_index]
    y_train, y_test = y_new[train_index], y_new[test_index]

    rf_classifier.fit(X_train, y_train)
    feature_importances_list.append(rf_classifier.feature_importances_)
average_feature_importances = np.mean(feature_importances_list, axis=0)
feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': average_feature_importances})


feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

print(feature_importance_df)

# Validate models for old data on new data

In [ ]:
## Validate model for old data on new data ##

balance = {0: 1, 1: 200}
X_train, X_test, y_train, y_test = X_old, X_new, y_old, y_new

# Create a Random Forest Classifier
rf_classifier = RandomForestClassifier(max_depth = 2, n_estimators = 1000, class_weight=balance, random_state = 10)

# Train the model
rf_classifier.fit(X_train, y_train)
RF_prd=rf_classifier.predict(X_test)
tn, fp, fn, tp = confusion_matrix(y_test, RF_prd).ravel()
ba = 1./2*(1.*tp/(tp+fn)+1.*tn/(tn+fp))
spe = 1.*tn/(tn+fp)
sens = 1.*tp/(tp+fn)
pres = tp/(tp+fp)
npvs = tn/(tn+fn)
cos = 565.*fn + 1.*fp
lis=[ba,spe,sens,pres,npvs,cos]
lis


In [ ]:
## Validate model for old data on new data ##

balance = {0: 1, 1: 300}
X_train, X_test, y_train, y_test = X_old, X_new, y_old, y_new

DT = DecisionTreeClassifier(max_depth=4, class_weight=balance, random_state = 10)
                
# Train the model
DT.fit(X_train, y_train)
DT_prd=DT.predict(X_test)
tn, fp, fn, tp = confusion_matrix(y_test, DT_prd).ravel()
ba = 1./2*(1.*tp/(tp+fn)+1.*tn/(tn+fp))
spe = 1.*tn/(tn+fp)
sens = 1.*tp/(tp+fn)
pres = tp/(tp+fp)
npvs = tn/(tn+fn)
cos = 565.*fn + 1.*fp
lis=[ba,spe,sens,pres,npvs,cos]
lis

In [ ]:
## Validate model for old data on new data ##

balance = {0: 1, 1: 300}
X_train, X_test, y_train, y_test = X_old, X_new, y_old, y_new

sv = SVC(C = 0.01, kernel = 'rbf', class_weight=balance, random_state = 10)
                
# Train the model
sv.fit(X_train, y_train)
sv_prd=sv.predict(X_test)
tn, fp, fn, tp = confusion_matrix(y_test, sv_prd).ravel()
ba = 1./2*(1.*tp/(tp+fn)+1.*tn/(tn+fp))
spe = 1.*tn/(tn+fp)
sens = 1.*tp/(tp+fn)
pres = tp/(tp+fp)
npvs = tn/(tn+fn)
cos = 565.*fn + 1.*fp
lis=[ba,spe,sens,pres,npvs,cos]
lis

In [ ]:
## Validate model for old data on new data ##

balance = {0: 1, 1: 300}
X_train, X_test, y_train, y_test = X_old, X_new, y_old, y_new

LogReg1 = LogisticRegression(C=0.01, penalty='l1', solver='liblinear', class_weight=balance, random_state = 10)
                
# Train the model
LogReg1.fit(X_train, y_train)
LogReg1_prd=LogReg1.predict(X_test)
tn, fp, fn, tp = confusion_matrix(y_test, LogReg1_prd).ravel()
ba = 1./2*(1.*tp/(tp+fn)+1.*tn/(tn+fp))
spe = 1.*tn/(tn+fp)
sens = 1.*tp/(tp+fn)
pres = tp/(tp+fp)
npvs = tn/(tn+fn)
cos = 565.*fn + 1.*fp
lis=[ba,spe,sens,pres,npvs,cos]
lis

In [ ]:
## Validate model for old data on new data ##

balance = {0: 1, 1: 300}
X_train, X_test, y_train, y_test = X_old, X_new, y_old, y_new

LogReg2 = LogisticRegression(C=0.01, penalty='l2', solver='liblinear', class_weight=balance, random_state = 10)
                
# Train the model
LogReg2.fit(X_train, y_train)
LogReg2_prd=LogReg2.predict(X_test)
tn, fp, fn, tp = confusion_matrix(y_test, LogReg2_prd).ravel()
ba = 1./2*(1.*tp/(tp+fn)+1.*tn/(tn+fp))
spe = 1.*tn/(tn+fp)
sens = 1.*tp/(tp+fn)
pres = tp/(tp+fp)
npvs = tn/(tn+fn)
cos = 565.*fn + 1.*fp
lis=[ba,spe,sens,pres,npvs,cos]
lis

In [ ]:
## StratifiedKFold cross validation ##

balances = [{0: 1, 1: 1}, {0: 1, 1: 200}, {0: 1, 1: 300}, {0: 1, 1: 500}, {0: 1, 1: 1000}]
m_dep = [2, 4, 6, 8, 10, 50, 100]
n_est = [5, 10, 100, 500, 1000]
#m_fea = ['sqrt','log2']
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=10)
#main loop for tuning
rf_n_result_dfs = []
for balance in balances:
    print('I am bal')
    lis = []
    for i in m_dep:
        for j in n_est:
            lst_bala_accu_stratified = []
            lst_spe_stratified = []
            lst_sens_stratified = []
            lst_exp_cost = []
            lst_pre = []
            lst_npv = []
            for train_index, test_index in skf.split(scaled_n, y_new):
                X_train_fold, X_test_fold = scaled_n[train_index], scaled_n[test_index]
                y_train_fold, y_test_fold = y_new[train_index], y_new[test_index]
                RF_cn = RandomForestClassifier(max_depth = i, n_estimators = j, class_weight=balance)
                RF_cn.fit(X_train_fold, y_train_fold)
                RF_cn_prd=RF_cn.predict(X_test_fold)
                tn, fp, fn, tp = confusion_matrix(y_test_fold, RF_cn_prd).ravel()
                ba = 1./2*(1.*tp/(tp+fn)+1.*tn/(tn+fp))
                spe = 1.*tn/(tn+fp)
                sens = 1.*tp/(tp+fn)
                pres = tp/(tp+fp)
                npvs = tn/(tn+fn)
                cos = 565.*fn + 1.*fp
                lst_bala_accu_stratified.append(ba)
                lst_spe_stratified.append(spe)
                lst_sens_stratified.append(sens)
                lst_exp_cost.append(cos)
                lst_pre.append(pres)
                lst_npv.append(npvs)
            #print(np.mean(lst_bala_accu_stratified))
            lis.append([balance,i,j,np.mean(lst_spe_stratified),np.mean(lst_sens_stratified),np.mean(lst_bala_accu_stratified),
                        np.mean(lst_pre),np.mean(lst_npv),np.mean(lst_exp_cost)])
    lis_dfNps = pd.DataFrame(lis)
    df_rf_baNps = lis_dfNps.sort_values(by=[5], ascending=False)
    rf_n_result_dfs.append(df_rf_baNps.head(1))
final_Nresult_df = pd.concat(rf_n_result_dfs, ignore_index=True)
#df_rf_pc5p = lis_df5p.sort_values(by=[5], ascending=False)
#print(df_rf_pc5p.head(1))

In [ ]:
final_Nresult_df = pd.concat(rf_n_result_dfs, ignore_index=True)
final_Nresult_df

In [218]:
final_Nresult_df['f1'] = 2* final_Nresult_df[4] * final_Nresult_df[6]/(final_Nresult_df[4] + final_Nresult_df[6])
final_Nresult_df['cost'] = final_Nresult_df[8] * 36

In [15]:
from sklearn.svm import SVC

In [ ]:
## StratifiedKFold cross validation ##

balances = [{0: 1, 1: 1}, {0: 1, 1: 200}, {0: 1, 1: 300}, {0: 1, 1: 500}, {0: 1, 1: 1000}]
c = [50, 10, 1.0, 0.1, 0.01]
ker = ['poly', 'rbf', 'sigmoid']

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=10)
#main loop for tuning
sv_o_result_dfs = []
for balance in balances:
    lis = []
    for i in c:
        for j in ker:
            lst_bala_accu_stratified = []
            lst_spe_stratified = []
            lst_sens_stratified = []
            lst_exp_cost = []
            lst_pre = []
            lst_npv = []
            for train_index, test_index in skf.split(scaled_o, y_old):
                X_train_fold, X_test_fold = scaled_o[train_index], scaled_o[test_index]
                y_train_fold, y_test_fold = y_old[train_index], y_old[test_index]
                sv = SVC(C = i, kernel = j, class_weight=balance)
                sv.fit(X_train_fold, y_train_fold)
                sv_prd=sv.predict(X_test_fold)
                tn, fp, fn, tp = confusion_matrix(y_test_fold, sv_prd).ravel()
                ba = 1./2*(1.*tp/(tp+fn)+1.*tn/(tn+fp))
                spe = 1.*tn/(tn+fp)
                sens = 1.*tp/(tp+fn)
                pres = tp/(tp+fp)
                npvs = tn/(tn+fn)
                cos = 565.*fn + 1.*fp
                lst_bala_accu_stratified.append(ba)
                lst_spe_stratified.append(spe)
                lst_sens_stratified.append(sens)
                lst_exp_cost.append(cos)
                lst_pre.append(pres)
                lst_npv.append(npvs)
            #print(np.mean(lst_bala_accu_stratified))
            lis.append([balance,i,j,np.mean(lst_spe_stratified),np.mean(lst_sens_stratified),np.mean(lst_bala_accu_stratified),
                        np.mean(lst_pre),np.mean(lst_npv),np.mean(lst_exp_cost)])
    lis_svo = pd.DataFrame(lis)
    df_svo = lis_svo.sort_values(by=[5], ascending=False)
    sv_o_result_dfs.append(df_svo.head(1))

In [ ]:
final_svo_result_df = pd.concat(sv_o_result_dfs, ignore_index=True)
final_svo_result_df

In [214]:
final_svo_result_df['f1'] = 2* final_svo_result_df[4] * final_svo_result_df[6]/(final_svo_result_df[4] + final_svo_result_df[6])
final_svo_result_df['cost'] = final_svo_result_df[8] * 36

In [ ]:
## StratifiedKFold cross validation ##

balances = [{0: 1, 1: 1}, {0: 1, 1: 200}, {0: 1, 1: 300}, {0: 1, 1: 500}, {0: 1, 1: 1000}]
c = [50, 10, 1.0, 0.1, 0.01]
ker = ['poly', 'rbf', 'sigmoid']
#StratifiedKFold cross validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=10)
#main loop for tuning
sv_n_result_dfs = []
for balance in balances:
    lis = []
    for i in c:
        for j in ker:
            lst_bala_accu_stratified = []
            lst_spe_stratified = []
            lst_sens_stratified = []
            lst_exp_cost = []
            lst_pre = []
            lst_npv = []
            for train_index, test_index in skf.split(scaled_n, y_new):
                X_train_fold, X_test_fold = scaled_n[train_index], scaled_n[test_index]
                y_train_fold, y_test_fold = y_new[train_index], y_new[test_index]
                sv = SVC(C = i, kernel = j, class_weight=balance)
                sv.fit(X_train_fold, y_train_fold)
                sv_prd=sv.predict(X_test_fold)
                tn, fp, fn, tp = confusion_matrix(y_test_fold, sv_prd).ravel()
                ba = 1./2*(1.*tp/(tp+fn)+1.*tn/(tn+fp))
                spe = 1.*tn/(tn+fp)
                sens = 1.*tp/(tp+fn)
                pres = tp/(tp+fp)
                npvs = tn/(tn+fn)
                cos = 565.*fn + 1.*fp
                lst_bala_accu_stratified.append(ba)
                lst_spe_stratified.append(spe)
                lst_sens_stratified.append(sens)
                lst_exp_cost.append(cos)
                lst_pre.append(pres)
                lst_npv.append(npvs)
            #print(np.mean(lst_bala_accu_stratified))
            lis.append([balance,i,j,np.mean(lst_spe_stratified),np.mean(lst_sens_stratified),np.mean(lst_bala_accu_stratified),
                        np.mean(lst_pre),np.mean(lst_npv),np.mean(lst_exp_cost)])
    lis_svn = pd.DataFrame(lis)
    df_svn = lis_svn.sort_values(by=[5], ascending=False)
    sv_n_result_dfs.append(df_svn.head(1))
final_svn_result_df = pd.concat(sv_n_result_dfs, ignore_index=True)

In [220]:
final_svn_result_df['f1'] = 2* final_svn_result_df[4] * final_svn_result_df[6]/(final_svn_result_df[4] + final_svn_result_df[6])
final_svn_result_df['cost'] = final_svn_result_df[8] * 36

In [ ]:
## StratifiedKFold cross validation ##

balances = [{0: 1, 1: 1}, {0: 1, 1: 200}, {0: 1, 1: 300}, {0: 1, 1: 500}, {0: 1, 1: 1000}]
m_dep = [2, 4, 6, 8, 10, 50, 100]
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=10)
dt_o_result_dfs = []
for balance in balances:
    lis = []
    for i in m_dep:
        lst_bala_accu_stratified = []
        lst_spe_stratified = []
        lst_sens_stratified = []
        lst_exp_cost = []
        lst_pre = []
        lst_npv = []
        for train_index, test_index in skf.split(scaled_o, y_old):
                X_train_fold, X_test_fold = scaled_o[train_index], scaled_o[test_index]
                y_train_fold, y_test_fold = y_old[train_index], y_old[test_index]
                DT = DecisionTreeClassifier(max_depth=i, class_weight=balance)
                DT.fit(X_train_fold, y_train_fold)
                DT_prd=DT.predict(X_test_fold)
                tn, fp, fn, tp = confusion_matrix(y_test_fold, DT_prd).ravel()
                ba = 1./2*(1.*tp/(tp+fn)+1.*tn/(tn+fp))
                spe = 1.*tn/(tn+fp)
                sens = 1.*tp/(tp+fn)
                pres = tp/(tp+fp)
                npvs = tn/(tn+fn)
                cos = 565.*fn + 1.*fp
                lst_bala_accu_stratified.append(ba)
                lst_spe_stratified.append(spe)
                lst_sens_stratified.append(sens)
                lst_exp_cost.append(cos)
                lst_pre.append(pres)
                lst_npv.append(npvs)
        #print(np.mean(lst_bala_accu_stratified))
        lis.append([balance,i,np.mean(lst_spe_stratified),np.mean(lst_sens_stratified),np.mean(lst_bala_accu_stratified),
                        np.mean(lst_pre),np.mean(lst_npv),np.mean(lst_exp_cost)])
    lis_dto = pd.DataFrame(lis)
    df_dto = lis_dto.sort_values(by=[4], ascending=False)
    dt_o_result_dfs.append(df_dto.head(1))
final_dto_result_df = pd.concat(dt_o_result_dfs, ignore_index=True)

In [210]:
final_dto_result_df['f1'] = 2* final_dto_result_df[5] * final_dto_result_df[3]/(final_dto_result_df[5] + final_dto_result_df[3])
final_dto_result_df['cost'] = final_dto_result_df[7] * 36

In [ ]:
## StratifiedKFold cross validation ##

balances = [{0: 1, 1: 1}, {0: 1, 1: 200}, {0: 1, 1: 300}, {0: 1, 1: 500}, {0: 1, 1: 1000}]
m_dep = [2, 4, 6, 8, 10, 50, 100]

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=10)
dt_n_result_dfs = []
for balance in balances:
    lis = []
    for i in m_dep:
        lst_bala_accu_stratified = []
        lst_spe_stratified = []
        lst_sens_stratified = []
        lst_exp_cost = []
        lst_pre = []
        lst_npv = []
        for train_index, test_index in skf.split(scaled_n, y_new):
                X_train_fold, X_test_fold = scaled_n[train_index], scaled_n[test_index]
                y_train_fold, y_test_fold = y_new[train_index], y_new[test_index]
                DT = DecisionTreeClassifier(max_depth=i, class_weight=balance)
                DT.fit(X_train_fold, y_train_fold)
                DT_prd=DT.predict(X_test_fold)
                tn, fp, fn, tp = confusion_matrix(y_test_fold, DT_prd).ravel()
                ba = 1./2*(1.*tp/(tp+fn)+1.*tn/(tn+fp))
                spe = 1.*tn/(tn+fp)
                sens = 1.*tp/(tp+fn)
                pres = tp/(tp+fp)
                npvs = tn/(tn+fn)
                cos = 565.*fn + 1.*fp
                lst_bala_accu_stratified.append(ba)
                lst_spe_stratified.append(spe)
                lst_sens_stratified.append(sens)
                lst_exp_cost.append(cos)
                lst_pre.append(pres)
                lst_npv.append(npvs)
        #print(np.mean(lst_bala_accu_stratified))
        lis.append([balance,i,np.mean(lst_spe_stratified),np.mean(lst_sens_stratified),np.mean(lst_bala_accu_stratified),
                        np.mean(lst_pre),np.mean(lst_npv),np.mean(lst_exp_cost)])
    lis_dtn = pd.DataFrame(lis)
    df_dtn = lis_dtn.sort_values(by=[4], ascending=False)
    dt_n_result_dfs.append(df_dtn.head(1))
final_dtn_result_df = pd.concat(dt_n_result_dfs, ignore_index=True)

In [222]:
final_dtn_result_df['f1'] = 2* final_dtn_result_df[5] * final_dtn_result_df[3]/(final_dtn_result_df[5] + final_dtn_result_df[3])
final_dtn_result_df['cost'] = final_dtn_result_df[7] * 36

In [ ]:
## StratifiedKFold cross validation ##

balances = [{0: 1, 1: 1}, {0: 1, 1: 200}, {0: 1, 1: 300}, {0: 1, 1: 500}, {0: 1, 1: 1000}]
c = [0.01, 0.1, 1, 10, 100]
sol = ['liblinear','saga']
lis = []
#StratifiedKFold cross validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=10)
#main loop for tuning
l1_o_result_dfs = []
for balance in balances:
    lis = []
    for i in c:
        for j in sol:
            lst_bala_accu_stratified = []
            lst_spe_stratified = []
            lst_sens_stratified = []
            lst_exp_cost = []
            lst_pre = []
            lst_npv = []
            for train_index, test_index in skf.split(scaled_o, y_old):
                X_train_fold, X_test_fold = scaled_o[train_index], scaled_o[test_index]
                y_train_fold, y_test_fold = y_old[train_index], y_old[test_index]
                LogReg1 = LogisticRegression(C=i, penalty='l1', solver=j, class_weight=balance)
                LogReg1.fit(X_train_fold, y_train_fold)
                LogReg1_prd=LogReg1.predict(X_test_fold)
                tn, fp, fn, tp = confusion_matrix(y_test_fold, LogReg1_prd).ravel()
                ba = 1./2*(1.*tp/(tp+fn)+1.*tn/(tn+fp))
                spe = 1.*tn/(tn+fp)
                sens = 1.*tp/(tp+fn)
                pres = tp/(tp+fp)
                npvs = tn/(tn+fn)
                cos = 565.*fn + 1.*fp
                lst_bala_accu_stratified.append(ba)
                lst_spe_stratified.append(spe)
                lst_sens_stratified.append(sens)
                lst_exp_cost.append(cos)
                lst_pre.append(pres)
                lst_npv.append(npvs)
            #print(np.mean(lst_bala_accu_stratified))
            lis.append([balance,i,j,np.mean(lst_spe_stratified),np.mean(lst_sens_stratified),np.mean(lst_bala_accu_stratified),
                        np.mean(lst_pre),np.mean(lst_npv),np.mean(lst_exp_cost)])
    lis_lr1o = pd.DataFrame(lis)
    df_lr1o = lis_lr1o.sort_values(by=[5], ascending=False)
    l1_o_result_dfs.append(df_lr1o.head(1))
final_lr1o_result_df = pd.concat(l1_o_result_dfs, ignore_index=True)

In [205]:
final_lr1o_result_df['f1'] = 2* final_lr1o_result_df[6] * final_lr1o_result_df[4]/(final_lr1o_result_df[6] + final_lr1o_result_df[4])
final_lr1o_result_df['cost'] = final_lr1o_result_df[8] * 36

In [ ]:
## StratifiedKFold cross validation ##

balances = [{0: 1, 1: 1}, {0: 1, 1: 200}, {0: 1, 1: 300}, {0: 1, 1: 500}, {0: 1, 1: 1000}]
c = [0.01, 0.1, 1, 10, 100]
sol = ['liblinear','saga']
lis = []
#StratifiedKFold cross validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=10)
#main loop for tuning
l1_n_result_dfs = []
for balance in balances:
    lis = []
    for i in c:
        for j in sol:
            lst_bala_accu_stratified = []
            lst_spe_stratified = []
            lst_sens_stratified = []
            lst_exp_cost = []
            lst_pre = []
            lst_npv = []
            for train_index, test_index in skf.split(scaled_n, y_new):
                X_train_fold, X_test_fold = scaled_n[train_index], scaled_n[test_index]
                y_train_fold, y_test_fold = y_new[train_index], y_new[test_index]
                LogReg1 = LogisticRegression(C=i, penalty='l1', solver=j, class_weight=balance)
                LogReg1.fit(X_train_fold, y_train_fold)
                LogReg1_prd=LogReg1.predict(X_test_fold)
                tn, fp, fn, tp = confusion_matrix(y_test_fold, LogReg1_prd).ravel()
                ba = 1./2*(1.*tp/(tp+fn)+1.*tn/(tn+fp))
                spe = 1.*tn/(tn+fp)
                sens = 1.*tp/(tp+fn)
                pres = tp/(tp+fp)
                npvs = tn/(tn+fn)
                cos = 565.*fn + 1.*fp
                lst_bala_accu_stratified.append(ba)
                lst_spe_stratified.append(spe)
                lst_sens_stratified.append(sens)
                lst_exp_cost.append(cos)
                lst_pre.append(pres)
                lst_npv.append(npvs)
            #print(np.mean(lst_bala_accu_stratified))
            lis.append([balance,i,j,np.mean(lst_spe_stratified),np.mean(lst_sens_stratified),np.mean(lst_bala_accu_stratified),
                        np.mean(lst_pre),np.mean(lst_npv),np.mean(lst_exp_cost)])
    lis_lr1n = pd.DataFrame(lis)
    df_lr1n = lis_lr1n.sort_values(by=[5], ascending=False)
    l1_n_result_dfs.append(df_lr1n.head(1))
final_lr1n_result_df = pd.concat(l1_n_result_dfs, ignore_index=True)

In [224]:
final_lr1n_result_df['f1'] = 2* final_lr1n_result_df[6] * final_lr1n_result_df[4]/(final_lr1n_result_df[6] + final_lr1n_result_df[4])
final_lr1n_result_df['cost'] = final_lr1n_result_df[8] * 36

In [ ]:
## StratifiedKFold cross validation ##

balances = [{0: 1, 1: 1}, {0: 1, 1: 200}, {0: 1, 1: 300}, {0: 1, 1: 500}, {0: 1, 1: 1000}]
c = [0.01, 0.1, 1, 10, 100]
sol = ['newton-cg', 'lbfgs', 'liblinear']
lis = []
#StratifiedKFold cross validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=10)
#main loop for tuning
l2_o_result_dfs = []
for balance in balances:
    lis = []
    for i in c:
        for j in sol:
            lst_bala_accu_stratified = []
            lst_spe_stratified = []
            lst_sens_stratified = []
            lst_exp_cost = []
            lst_pre = []
            lst_npv = []
            for train_index, test_index in skf.split(scaled_o, y_old):
                X_train_fold, X_test_fold = scaled_o[train_index], scaled_o[test_index]
                y_train_fold, y_test_fold = y_old[train_index], y_old[test_index]
                LogReg2 = LogisticRegression(C=i, penalty='l2', solver=j, class_weight=balance)
                LogReg2.fit(X_train_fold, y_train_fold)
                LogReg2_prd=LogReg2.predict(X_test_fold)
                tn, fp, fn, tp = confusion_matrix(y_test_fold, LogReg2_prd).ravel()
                ba = 1./2*(1.*tp/(tp+fn)+1.*tn/(tn+fp))
                spe = 1.*tn/(tn+fp)
                sens = 1.*tp/(tp+fn)
                pres = tp/(tp+fp)
                npvs = tn/(tn+fn)
                cos = 565.*fn + 1.*fp
                lst_bala_accu_stratified.append(ba)
                lst_spe_stratified.append(spe)
                lst_sens_stratified.append(sens)
                lst_exp_cost.append(cos)
                lst_pre.append(pres)
                lst_npv.append(npvs)
            #print(np.mean(lst_bala_accu_stratified))
            lis.append([balance,i,j,np.mean(lst_spe_stratified),np.mean(lst_sens_stratified),np.mean(lst_bala_accu_stratified),
                        np.mean(lst_pre),np.mean(lst_npv),np.mean(lst_exp_cost)])
    lis_lr2o = pd.DataFrame(lis)
    df_lr2o = lis_lr2o.sort_values(by=[5], ascending=False)
    l2_o_result_dfs.append(df_lr2o.head(1))
final_lr2o_result_df = pd.concat(l2_o_result_dfs, ignore_index=True)

In [201]:
final_lr2o_result_df['f1'] = 2* final_lr2o_result_df[6] * final_lr2o_result_df[4]/(final_lr2o_result_df[6] + final_lr2o_result_df[4])

In [203]:
final_lr2o_result_df['cost'] = final_lr2o_result_df[8] * 36

In [ ]:
## StratifiedKFold cross validation ##

balances = [{0: 1, 1: 1}, {0: 1, 1: 200}, {0: 1, 1: 300}, {0: 1, 1: 500}, {0: 1, 1: 1000}]
c = [0.01, 0.1, 1, 10, 100]
sol = ['newton-cg', 'lbfgs', 'liblinear']
lis = []
#StratifiedKFold cross validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=10)
#main loop for tuning
l2_n_result_dfs = []
for balance in balances:
    lis = []
    for i in c:
        for j in sol:
            lst_bala_accu_stratified = []
            lst_spe_stratified = []
            lst_sens_stratified = []
            lst_exp_cost = []
            lst_pre = []
            lst_npv = []
            for train_index, test_index in skf.split(scaled_n, y_new):
                X_train_fold, X_test_fold = scaled_n[train_index], scaled_n[test_index]
                y_train_fold, y_test_fold = y_new[train_index], y_new[test_index]
                LogReg2 = LogisticRegression(C=i, penalty='l2', solver=j, class_weight=balance)
                LogReg2.fit(X_train_fold, y_train_fold)
                LogReg2_prd=LogReg2.predict(X_test_fold)
                tn, fp, fn, tp = confusion_matrix(y_test_fold, LogReg2_prd).ravel()
                ba = 1./2*(1.*tp/(tp+fn)+1.*tn/(tn+fp))
                spe = 1.*tn/(tn+fp)
                sens = 1.*tp/(tp+fn)
                pres = tp/(tp+fp)
                npvs = tn/(tn+fn)
                cos = 565.*fn + 1.*fp
                lst_bala_accu_stratified.append(ba)
                lst_spe_stratified.append(spe)
                lst_sens_stratified.append(sens)
                lst_exp_cost.append(cos)
                lst_pre.append(pres)
                lst_npv.append(npvs)
            #print(np.mean(lst_bala_accu_stratified))
            lis.append([balance,i,j,np.mean(lst_spe_stratified),np.mean(lst_sens_stratified),np.mean(lst_bala_accu_stratified),
                        np.mean(lst_pre),np.mean(lst_npv),np.mean(lst_exp_cost)])
    lis_lr2n = pd.DataFrame(lis)
    df_lr2n = lis_lr2n.sort_values(by=[5], ascending=False)
    l2_n_result_dfs.append(df_lr2n.head(1))
final_lr2n_result_df = pd.concat(l2_n_result_dfs, ignore_index=True)

In [226]:
final_lr2n_result_df['f1'] = 2* final_lr2n_result_df[6] * final_lr2n_result_df[4]/(final_lr2n_result_df[6] + final_lr2n_result_df[4])
final_lr2n_result_df['cost'] = final_lr2n_result_df[8] * 36

In [ ]:
## StratifiedKFold cross validation ##

balance = {0:1,1:500}
m_dep = [2, 4, 6, 8, 10, 50, 100]
n_est = [5, 10, 100, 500, 1000]
#m_fea = ['sqrt','log2']
lis = []
#StratifiedKFold cross validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=10)
#main loop for tuning
for i in m_dep:
    for j in n_est:
        lst_bala_accu_stratified = []
        lst_spe_stratified = []
        lst_sens_stratified = []
        lst_exp_cost = []
        lst_pre = []
        lst_npv = []
        for train_index, test_index in skf.split(scaled, y):
            X_train_fold, X_test_fold = scaled[train_index], scaled[test_index]
            y_train_fold, y_test_fold = y[train_index], y[test_index]
            RF_c5 = RandomForestClassifier(max_depth = i, n_estimators = j, class_weight=balance)
            RF_c5.fit(X_train_fold, y_train_fold)
            RF_c5_prd=RF_c5.predict(X_test_fold)
            tn, fp, fn, tp = confusion_matrix(y_test_fold, RF_c5_prd).ravel()
            ba = 1./2*(1.*tp/(tp+fn)+1.*tn/(tn+fp))
            spe = 1.*tn/(tn+fp)
            sens = 1.*tp/(tp+fn)
            pres = tp/(tp+fp)
            npvs = tn/(tn+fn)
            cos = 565.*fn + 1.*fp
            lst_bala_accu_stratified.append(ba)
            lst_spe_stratified.append(spe)
            lst_sens_stratified.append(sens)
            lst_exp_cost.append(cos)
            lst_pre.append(pres)
            lst_npv.append(npvs)
        #print(np.mean(lst_bala_accu_stratified))
        lis.append([i,j,np.mean(lst_spe_stratified),np.mean(lst_sens_stratified),np.mean(lst_bala_accu_stratified),
                    np.mean(lst_pre),np.mean(lst_npv),np.mean(lst_exp_cost)])
lis_df5ps = pd.DataFrame(lis)
df_rf_ba5ps = lis_df5ps.sort_values(by=[4], ascending=False)
#df_rf_pc5p = lis_df5p.sort_values(by=[5], ascending=False)
print(df_rf_ba5ps.head(1))
#print(df_rf_pc5p.head(1))

In [ ]:
## StratifiedKFold cross validation ##

balance = {0:1,1:200}
m_dep = [2, 4, 6, 8, 10, 50, 100]
n_est = [5, 10, 100, 500, 1000]
#m_fea = ['sqrt','log2']
lis = []
#StratifiedKFold cross validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=10)
#main loop for tuning
for i in m_dep:
    for j in n_est:
        lst_bala_accu_stratified = []
        lst_spe_stratified = []
        lst_sens_stratified = []
        lst_exp_cost = []
        lst_pre = []
        lst_npv = []
        for train_index, test_index in skf.split(scaled, y):
            X_train_fold, X_test_fold = scaled[train_index], scaled[test_index]
            y_train_fold, y_test_fold = y[train_index], y[test_index]
            RF_c5 = RandomForestClassifier(max_depth = i, n_estimators = j, class_weight=balance)
            RF_c5.fit(X_train_fold, y_train_fold)
            RF_c5_prd=RF_c5.predict(X_test_fold)
            tn, fp, fn, tp = confusion_matrix(y_test_fold, RF_c5_prd).ravel()
            ba = 1./2*(1.*tp/(tp+fn)+1.*tn/(tn+fp))
            spe = 1.*tn/(tn+fp)
            sens = 1.*tp/(tp+fn)
            pres = tp/(tp+fp)
            npvs = tn/(tn+fn)
            cos = 565.*fn + 1.*fp
            lst_bala_accu_stratified.append(ba)
            lst_spe_stratified.append(spe)
            lst_sens_stratified.append(sens)
            lst_exp_cost.append(cos)
            lst_pre.append(pres)
            lst_npv.append(npvs)
        #print(np.mean(lst_bala_accu_stratified))
        lis.append([i,j,np.mean(lst_spe_stratified),np.mean(lst_sens_stratified),np.mean(lst_bala_accu_stratified),
                    np.mean(lst_pre),np.mean(lst_npv),np.mean(lst_exp_cost)])
lis_df2ps = pd.DataFrame(lis)
df_rf_ba2ps = lis_df2ps.sort_values(by=[4], ascending=False)
#df_rf_pc2p = lis_df2p.sort_values(by=[5], ascending=False)
print(df_rf_ba2ps.head(1))
#print(df_rf_pc2p.head(1))

In [ ]:
## StratifiedKFold cross validation ##

balance = {0:1,1:300}
m_dep = [2, 4, 6, 8, 10, 50, 100]
n_est = [5, 10, 100, 500, 1000]
#m_fea = ['sqrt','log2']
lis = []
#StratifiedKFold cross validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=10)
#main loop for tuning
for i in m_dep:
    for j in n_est:
        lst_bala_accu_stratified = []
        lst_spe_stratified = []
        lst_sens_stratified = []
        lst_exp_cost = []
        lst_pre = []
        lst_npv = []
        for train_index, test_index in skf.split(scaled, y):
            X_train_fold, X_test_fold = scaled[train_index], scaled[test_index]
            y_train_fold, y_test_fold = y[train_index], y[test_index]
            RF_c5 = RandomForestClassifier(max_depth = i, n_estimators = j, class_weight=balance)
            RF_c5.fit(X_train_fold, y_train_fold)
            RF_c5_prd=RF_c5.predict(X_test_fold)
            tn, fp, fn, tp = confusion_matrix(y_test_fold, RF_c5_prd).ravel()
            ba = 1./2*(1.*tp/(tp+fn)+1.*tn/(tn+fp))
            spe = 1.*tn/(tn+fp)
            sens = 1.*tp/(tp+fn)
            pres = tp/(tp+fp)
            npvs = tn/(tn+fn)
            cos = 565.*fn + 1.*fp
            lst_bala_accu_stratified.append(ba)
            lst_spe_stratified.append(spe)
            lst_sens_stratified.append(sens)
            lst_exp_cost.append(cos)
            lst_pre.append(pres)
            lst_npv.append(npvs)
        #print(np.mean(lst_bala_accu_stratified))
        lis.append([i,j,np.mean(lst_spe_stratified),np.mean(lst_sens_stratified),np.mean(lst_bala_accu_stratified),
                    np.mean(lst_pre),np.mean(lst_npv),np.mean(lst_exp_cost)])
lis_df3ps = pd.DataFrame(lis)
df_rf_ba3ps = lis_df3ps.sort_values(by=[4], ascending=False)
#df_rf_pc2p = lis_df2p.sort_values(by=[5], ascending=False)
print(df_rf_ba3ps.head(1))
#print(df_rf_pc2p.head(1))

In [ ]:
## StratifiedKFold cross validation ##

balance = {0:1,1:1000}
m_dep = [2, 4, 6, 8, 10, 50, 100]
n_est = [5, 10, 100, 500, 1000]
#m_fea = ['sqrt','log2']
lis = []
#StratifiedKFold cross validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=10)
#main loop for tuning
for i in m_dep:
    for j in n_est:
        lst_bala_accu_stratified = []
        lst_spe_stratified = []
        lst_sens_stratified = []
        lst_exp_cost = []
        lst_pre = []
        lst_npv = []
        for train_index, test_index in skf.split(scaled, y):
            X_train_fold, X_test_fold = scaled[train_index], scaled[test_index]
            y_train_fold, y_test_fold = y[train_index], y[test_index]
            RF_c5 = RandomForestClassifier(max_depth = i, n_estimators = j, class_weight=balance)
            RF_c5.fit(X_train_fold, y_train_fold)
            RF_c5_prd=RF_c5.predict(X_test_fold)
            tn, fp, fn, tp = confusion_matrix(y_test_fold, RF_c5_prd).ravel()
            ba = 1./2*(1.*tp/(tp+fn)+1.*tn/(tn+fp))
            spe = 1.*tn/(tn+fp)
            sens = 1.*tp/(tp+fn)
            pres = tp/(tp+fp)
            npvs = tn/(tn+fn)
            cos = 565.*fn + 1.*fp
            lst_bala_accu_stratified.append(ba)
            lst_spe_stratified.append(spe)
            lst_sens_stratified.append(sens)
            lst_exp_cost.append(cos)
            lst_pre.append(pres)
            lst_npv.append(npvs)
        #print(np.mean(lst_bala_accu_stratified))
        lis.append([i,j,np.mean(lst_spe_stratified),np.mean(lst_sens_stratified),np.mean(lst_bala_accu_stratified),
                    np.mean(lst_pre),np.mean(lst_npv),np.mean(lst_exp_cost)])
lis_df10ps = pd.DataFrame(lis)
df_rf_ba10ps = lis_df10ps.sort_values(by=[4], ascending=False)
#df_rf_pc2p = lis_df2p.sort_values(by=[5], ascending=False)
print(df_rf_ba10ps.head(1))
#print(df_rf_pc2p.head(1))

In [ ]:
## StratifiedKFold cross validation ##

#balance = {0:1,1:200}
m_dep = [2, 4, 6, 8, 10, 50, 100]
n_est = [5, 10, 100, 500, 1000]
#m_fea = ['sqrt','log2']
lis = []
#StratifiedKFold cross validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=10)
#main loop for tuning
for i in m_dep:
    for j in n_est:
        lst_bala_accu_stratified = []
        lst_spe_stratified = []
        lst_sens_stratified = []
        lst_exp_cost = []
        lst_pre = []
        lst_npv = []
        for train_index, test_index in skf.split(scaled, y):
            X_train_fold, X_test_fold = scaled[train_index], scaled[test_index]
            y_train_fold, y_test_fold = y[train_index], y[test_index]
            RF_c5 = RandomForestClassifier(max_depth = i, n_estimators = j)
            RF_c5.fit(X_train_fold, y_train_fold)
            RF_c5_prd=RF_c5.predict(X_test_fold)
            tn, fp, fn, tp = confusion_matrix(y_test_fold, RF_c5_prd).ravel()
            ba = 1./2*(1.*tp/(tp+fn)+1.*tn/(tn+fp))
            spe = 1.*tn/(tn+fp)
            sens = 1.*tp/(tp+fn)
            pres = 1.*tp/(tp+fp)
            npvs = 1.*tn/(tn+fn)
            cos = 565.*fn + 1.*fp
            lst_bala_accu_stratified.append(ba)
            lst_spe_stratified.append(spe)
            lst_sens_stratified.append(sens)
            lst_exp_cost.append(cos)
            lst_pre.append(pres)
            lst_npv.append(npvs)
        #print(np.mean(lst_bala_accu_stratified))
        lis.append([i,j,np.mean(lst_spe_stratified),np.mean(lst_sens_stratified),np.mean(lst_bala_accu_stratified),
                    np.mean(lst_pre),np.mean(lst_npv),np.mean(lst_exp_cost)])
lis_df0ps = pd.DataFrame(lis)
df_rf_ba0ps = lis_df0ps.sort_values(by=[4], ascending=False)
#df_rf_pc2p = lis_df2p.sort_values(by=[5], ascending=False)
print(df_rf_ba0ps.head(1))
#print(df_rf_pc2p.head(1))

In [ ]:
## StratifiedKFold cross validation ##

balance = {0:1,1:300}
c = [0.01, 0.1, 1, 10, 100]
sol = ['newton-cg', 'lbfgs', 'liblinear']
lis = []
#StratifiedKFold cross validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=10)
#main loop for tuning
for i in c:
    for j in sol:
        lst_bala_accu_stratified = []
        lst_spe_stratified = []
        lst_sens_stratified = []
        lst_exp_cost = []
        lst_pre = []
        lst_npv = []
        for train_index, test_index in skf.split(scaled, y):
            X_train_fold, X_test_fold = scaled[train_index], scaled[test_index]
            y_train_fold, y_test_fold = y[train_index], y[test_index]
            LogReg1 = LogisticRegression(C=i, penalty='l2', solver=j, class_weight=balance)
            LogReg1.fit(X_train_fold, y_train_fold)
            LogReg1_prd=LogReg1.predict(X_test_fold)
            tn, fp, fn, tp = confusion_matrix(y_test_fold, LogReg1_prd).ravel()
            ba = 1./2*(1.*tp/(tp+fn)+1.*tn/(tn+fp))
            spe = 1.*tn/(tn+fp)
            sens = 1.*tp/(tp+fn)
            pres = tp/(tp+fp)
            npvs = tn/(tn+fn)
            cos = 565.*fn + 1.*fp
            lst_bala_accu_stratified.append(ba)
            lst_spe_stratified.append(spe)
            lst_sens_stratified.append(sens)
            lst_exp_cost.append(cos)
            lst_pre.append(pres)
            lst_npv.append(npvs)
        #print(np.mean(lst_bala_accu_stratified))
        lis.append([i,j,np.mean(lst_spe_stratified),np.mean(lst_sens_stratified),np.mean(lst_bala_accu_stratified),
                    np.mean(lst_pre),np.mean(lst_npv),np.mean(lst_exp_cost)])
lis_lr3ps = pd.DataFrame(lis)
df_lr_ba3ps = lis_lr3ps.sort_values(by=[4], ascending=False)
#df_lr_pc3p = lis_lr3p.sort_values(by=[5], ascending=False)
print(df_lr_ba3ps.head(1))
#print(df_lr_pc3p.head(1))

In [ ]:
## StratifiedKFold cross validation ##

balance = {0:1,1:200}
c = [0.01, 0.1, 1, 10, 100]
sol = ['newton-cg', 'lbfgs', 'liblinear']
lis = []
#StratifiedKFold cross validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=10)
#main loop for tuning
for i in c:
    for j in sol:
        lst_bala_accu_stratified = []
        lst_spe_stratified = []
        lst_sens_stratified = []
        lst_exp_cost = []
        lst_pre = []
        lst_npv = []
        for train_index, test_index in skf.split(scaled, y):
            X_train_fold, X_test_fold = scaled[train_index], scaled[test_index]
            y_train_fold, y_test_fold = y[train_index], y[test_index]
            LogReg1 = LogisticRegression(C=i, penalty='l2', solver=j, class_weight=balance)
            LogReg1.fit(X_train_fold, y_train_fold)
            LogReg1_prd=LogReg1.predict(X_test_fold)
            tn, fp, fn, tp = confusion_matrix(y_test_fold, LogReg1_prd).ravel()
            ba = 1./2*(1.*tp/(tp+fn)+1.*tn/(tn+fp))
            spe = 1.*tn/(tn+fp)
            sens = 1.*tp/(tp+fn)
            pres = tp/(tp+fp)
            npvs = tn/(tn+fn)
            cos = 565.*fn + 1.*fp
            lst_bala_accu_stratified.append(ba)
            lst_spe_stratified.append(spe)
            lst_sens_stratified.append(sens)
            lst_exp_cost.append(cos)
            lst_pre.append(pres)
            lst_npv.append(npvs)
        #print(np.mean(lst_bala_accu_stratified))
        lis.append([i,j,np.mean(lst_spe_stratified),np.mean(lst_sens_stratified),np.mean(lst_bala_accu_stratified),
                    np.mean(lst_pre),np.mean(lst_npv),np.mean(lst_exp_cost)])
lis_lr2ps = pd.DataFrame(lis)
df_lr_ba2ps = lis_lr2ps.sort_values(by=[4], ascending=False)
#df_lr_pc3p = lis_lr3p.sort_values(by=[5], ascending=False)
print(df_lr_ba2ps.head(1))
#print(df_lr_pc3p.head(1))

In [ ]:
## StratifiedKFold cross validation ##

balance = {0:1,1:500}
c = [0.01, 0.1, 1, 10, 100]
sol = ['newton-cg', 'lbfgs', 'liblinear']
lis = []
#StratifiedKFold cross validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=10)
#main loop for tuning
for i in c:
    for j in sol:
        lst_bala_accu_stratified = []
        lst_spe_stratified = []
        lst_sens_stratified = []
        lst_exp_cost = []
        lst_pre = []
        lst_npv = []
        for train_index, test_index in skf.split(scaled, y):
            X_train_fold, X_test_fold = scaled[train_index], scaled[test_index]
            y_train_fold, y_test_fold = y[train_index], y[test_index]
            LogReg1 = LogisticRegression(C=i, penalty='l2', solver=j, class_weight=balance)
            LogReg1.fit(X_train_fold, y_train_fold)
            LogReg1_prd=LogReg1.predict(X_test_fold)
            tn, fp, fn, tp = confusion_matrix(y_test_fold, LogReg1_prd).ravel()
            ba = 1./2*(1.*tp/(tp+fn)+1.*tn/(tn+fp))
            spe = 1.*tn/(tn+fp)
            sens = 1.*tp/(tp+fn)
            pres = tp/(tp+fp)
            npvs = tn/(tn+fn)
            cos = 565.*fn + 1.*fp
            lst_bala_accu_stratified.append(ba)
            lst_spe_stratified.append(spe)
            lst_sens_stratified.append(sens)
            lst_exp_cost.append(cos)
            lst_pre.append(pres)
            lst_npv.append(npvs)
        #print(np.mean(lst_bala_accu_stratified))
        lis.append([i,j,np.mean(lst_spe_stratified),np.mean(lst_sens_stratified),np.mean(lst_bala_accu_stratified),
                    np.mean(lst_pre),np.mean(lst_npv),np.mean(lst_exp_cost)])
lis_lr5ps = pd.DataFrame(lis)
df_lr_ba5ps = lis_lr5ps.sort_values(by=[4], ascending=False)
#df_lr_pc3p = lis_lr3p.sort_values(by=[5], ascending=False)
print(df_lr_ba5ps.head(1))
#print(df_lr_pc3p.head(1))

In [ ]:
## StratifiedKFold cross validation ##

balance = {0:1,1:1000}
c = [0.01, 0.1, 1, 10, 100]
sol = ['newton-cg', 'lbfgs', 'liblinear']
lis = []
#StratifiedKFold cross validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=10)
#main loop for tuning
for i in c:
    for j in sol:
        lst_bala_accu_stratified = []
        lst_spe_stratified = []
        lst_sens_stratified = []
        lst_exp_cost = []
        lst_pre = []
        lst_npv = []
        for train_index, test_index in skf.split(scaled, y):
            X_train_fold, X_test_fold = scaled[train_index], scaled[test_index]
            y_train_fold, y_test_fold = y[train_index], y[test_index]
            LogReg1 = LogisticRegression(C=i, penalty='l2', solver=j, class_weight=balance)
            LogReg1.fit(X_train_fold, y_train_fold)
            LogReg1_prd=LogReg1.predict(X_test_fold)
            tn, fp, fn, tp = confusion_matrix(y_test_fold, LogReg1_prd).ravel()
            ba = 1./2*(1.*tp/(tp+fn)+1.*tn/(tn+fp))
            spe = 1.*tn/(tn+fp)
            sens = 1.*tp/(tp+fn)
            pres = tp/(tp+fp)
            npvs = tn/(tn+fn)
            cos = 565.*fn + 1.*fp
            lst_bala_accu_stratified.append(ba)
            lst_spe_stratified.append(spe)
            lst_sens_stratified.append(sens)
            lst_exp_cost.append(cos)
            lst_pre.append(pres)
            lst_npv.append(npvs)
        #print(np.mean(lst_bala_accu_stratified))
        lis.append([i,j,np.mean(lst_spe_stratified),np.mean(lst_sens_stratified),np.mean(lst_bala_accu_stratified),
                    np.mean(lst_pre),np.mean(lst_npv),np.mean(lst_exp_cost)])
lis_lr10ps = pd.DataFrame(lis)
df_lr_ba10ps = lis_lr10ps.sort_values(by=[4], ascending=False)
#df_lr_pc3p = lis_lr3p.sort_values(by=[5], ascending=False)
print(df_lr_ba10ps.head(1))
#print(df_lr_pc3p.head(1))

In [ ]:
## StratifiedKFold cross validation ##

c = [0.01, 0.1, 1, 10, 100]
sol = ['newton-cg', 'lbfgs', 'liblinear']
lis = []
#StratifiedKFold cross validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=10)
#main loop for tuning
for i in c:
    for j in sol:
        lst_bala_accu_stratified = []
        lst_spe_stratified = []
        lst_sens_stratified = []
        lst_exp_cost = []
        lst_pre = []
        lst_npv = []
        for train_index, test_index in skf.split(scaled, y):
            X_train_fold, X_test_fold = scaled[train_index], scaled[test_index]
            y_train_fold, y_test_fold = y[train_index], y[test_index]
            LogReg1 = LogisticRegression(C=i, penalty='l2', solver=j)
            LogReg1.fit(X_train_fold, y_train_fold)
            LogReg1_prd=LogReg1.predict(X_test_fold)
            tn, fp, fn, tp = confusion_matrix(y_test_fold, LogReg1_prd).ravel()
            ba = 1./2*(1.*tp/(tp+fn)+1.*tn/(tn+fp))
            spe = 1.*tn/(tn+fp)
            sens = 1.*tp/(tp+fn)
            pres = tp/(tp+fp)
            npvs = tn/(tn+fn)
            cos = 565.*fn + 1.*fp
            lst_bala_accu_stratified.append(ba)
            lst_spe_stratified.append(spe)
            lst_sens_stratified.append(sens)
            lst_exp_cost.append(cos)
            lst_pre.append(pres)
            lst_npv.append(npvs)
        #print(np.mean(lst_bala_accu_stratified))
        lis.append([i,j,np.mean(lst_spe_stratified),np.mean(lst_sens_stratified),np.mean(lst_bala_accu_stratified),
                    np.mean(lst_pre),np.mean(lst_npv),np.mean(lst_exp_cost)])
lis_lr0ps = pd.DataFrame(lis)
df_lr_ba0ps = lis_lr0ps.sort_values(by=[4], ascending=False)
#df_lr_pc3p = lis_lr3p.sort_values(by=[5], ascending=False)
print(df_lr_ba0ps.head(1))
#print(df_lr_pc3p.head(1))

In [ ]:
## StratifiedKFold cross validation ##

balance = {0:1,1:1000}
c = [50, 10, 1.0, 0.1, 0.01]
ker = ['poly', 'rbf', 'sigmoid']
lis = []
#StratifiedKFold cross validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=10)
#main loop for tuning
for i in c:
    for j in ker:
        lst_bala_accu_stratified = []
        lst_spe_stratified = []
        lst_sens_stratified = []
        lst_exp_cost = []
        lst_pre = []
        lst_npv = []
        for train_index, test_index in skf.split(scaled, y):
            X_train_fold, X_test_fold = scaled[train_index], scaled[test_index]
            y_train_fold, y_test_fold = y[train_index], y[test_index]
            sv = SVC(C = i, kernel = j, class_weight=balance)
            sv.fit(X_train_fold, y_train_fold)
            sv_prd=sv.predict(X_test_fold)
            tn, fp, fn, tp = confusion_matrix(y_test_fold, sv_prd).ravel()
            ba = 1./2*(1.*tp/(tp+fn)+1.*tn/(tn+fp))
            spe = 1.*tn/(tn+fp)
            sens = 1.*tp/(tp+fn)
            pres = tp/(tp+fp)
            npvs = tn/(tn+fn)
            cos = 565.*fn + 1.*fp
            lst_bala_accu_stratified.append(ba)
            lst_spe_stratified.append(spe)
            lst_sens_stratified.append(sens)
            lst_exp_cost.append(cos)
            lst_pre.append(pres)
            lst_npv.append(npvs)
        #print(np.mean(lst_bala_accu_stratified))
        lis.append([i,j,np.mean(lst_spe_stratified),np.mean(lst_sens_stratified),np.mean(lst_bala_accu_stratified),
                    np.mean(lst_pre),np.mean(lst_npv),np.mean(lst_exp_cost)])
lis_sv10 = pd.DataFrame(lis)
df_sv10 = lis_sv10.sort_values(by=[4], ascending=False)
print(df_sv10.head(1))

In [ ]:
## StratifiedKFold cross validation ##

balance = {0:1,1:500}
c = [50, 10, 1.0, 0.1, 0.01]
ker = ['poly', 'rbf', 'sigmoid']
lis = []
#StratifiedKFold cross validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=10)
#main loop for tuning
for i in c:
    for j in ker:
        lst_bala_accu_stratified = []
        lst_spe_stratified = []
        lst_sens_stratified = []
        lst_exp_cost = []
        lst_pre = []
        lst_npv = []
        for train_index, test_index in skf.split(scaled, y):
            X_train_fold, X_test_fold = scaled[train_index], scaled[test_index]
            y_train_fold, y_test_fold = y[train_index], y[test_index]
            sv = SVC(C = i, kernel = j, class_weight=balance)
            sv.fit(X_train_fold, y_train_fold)
            sv_prd=sv.predict(X_test_fold)
            tn, fp, fn, tp = confusion_matrix(y_test_fold, sv_prd).ravel()
            ba = 1./2*(1.*tp/(tp+fn)+1.*tn/(tn+fp))
            spe = 1.*tn/(tn+fp)
            sens = 1.*tp/(tp+fn)
            pres = tp/(tp+fp)
            npvs = tn/(tn+fn)
            cos = 565.*fn + 1.*fp
            lst_bala_accu_stratified.append(ba)
            lst_spe_stratified.append(spe)
            lst_sens_stratified.append(sens)
            lst_exp_cost.append(cos)
            lst_pre.append(pres)
            lst_npv.append(npvs)
        #print(np.mean(lst_bala_accu_stratified))
        lis.append([i,j,np.mean(lst_spe_stratified),np.mean(lst_sens_stratified),np.mean(lst_bala_accu_stratified),
                    np.mean(lst_pre),np.mean(lst_npv),np.mean(lst_exp_cost)])
lis_sv5 = pd.DataFrame(lis)
df_sv5 = lis_sv5.sort_values(by=[4], ascending=False)
print(df_sv5.head(1))

In [ ]:
## StratifiedKFold cross validation ##

balance = {0:1,1:300}
c = [50, 10, 1.0, 0.1, 0.01]
ker = ['poly', 'rbf', 'sigmoid']
lis = []
#StratifiedKFold cross validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=10)
#main loop for tuning
for i in c:
    for j in ker:
        lst_bala_accu_stratified = []
        lst_spe_stratified = []
        lst_sens_stratified = []
        lst_exp_cost = []
        lst_pre = []
        lst_npv = []
        for train_index, test_index in skf.split(scaled, y):
            X_train_fold, X_test_fold = scaled[train_index], scaled[test_index]
            y_train_fold, y_test_fold = y[train_index], y[test_index]
            sv = SVC(C = i, kernel = j, class_weight=balance)
            sv.fit(X_train_fold, y_train_fold)
            sv_prd=sv.predict(X_test_fold)
            tn, fp, fn, tp = confusion_matrix(y_test_fold, sv_prd).ravel()
            ba = 1./2*(1.*tp/(tp+fn)+1.*tn/(tn+fp))
            spe = 1.*tn/(tn+fp)
            sens = 1.*tp/(tp+fn)
            pres = tp/(tp+fp)
            npvs = tn/(tn+fn)
            cos = 565.*fn + 1.*fp
            lst_bala_accu_stratified.append(ba)
            lst_spe_stratified.append(spe)
            lst_sens_stratified.append(sens)
            lst_exp_cost.append(cos)
            lst_pre.append(pres)
            lst_npv.append(npvs)
        #print(np.mean(lst_bala_accu_stratified))
        lis.append([i,j,np.mean(lst_spe_stratified),np.mean(lst_sens_stratified),np.mean(lst_bala_accu_stratified),
                    np.mean(lst_pre),np.mean(lst_npv),np.mean(lst_exp_cost)])
lis_sv3 = pd.DataFrame(lis)
df_sv3 = lis_sv3.sort_values(by=[4], ascending=False)
print(df_sv3.head(1))

In [ ]:
## StratifiedKFold cross validation ##

balance = {0:1,1:200}
c = [50, 10, 1.0, 0.1, 0.01]
ker = ['poly', 'rbf', 'sigmoid']
lis = []
#StratifiedKFold cross validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=10)
#main loop for tuning
for i in c:
    for j in ker:
        lst_bala_accu_stratified = []
        lst_spe_stratified = []
        lst_sens_stratified = []
        lst_exp_cost = []
        lst_pre = []
        lst_npv = []
        for train_index, test_index in skf.split(scaled, y):
            X_train_fold, X_test_fold = scaled[train_index], scaled[test_index]
            y_train_fold, y_test_fold = y[train_index], y[test_index]
            sv = SVC(C = i, kernel = j, class_weight=balance)
            sv.fit(X_train_fold, y_train_fold)
            sv_prd=sv.predict(X_test_fold)
            tn, fp, fn, tp = confusion_matrix(y_test_fold, sv_prd).ravel()
            ba = 1./2*(1.*tp/(tp+fn)+1.*tn/(tn+fp))
            spe = 1.*tn/(tn+fp)
            sens = 1.*tp/(tp+fn)
            pres = tp/(tp+fp)
            npvs = tn/(tn+fn)
            cos = 565.*fn + 1.*fp
            lst_bala_accu_stratified.append(ba)
            lst_spe_stratified.append(spe)
            lst_sens_stratified.append(sens)
            lst_exp_cost.append(cos)
            lst_pre.append(pres)
            lst_npv.append(npvs)
        #print(np.mean(lst_bala_accu_stratified))
        lis.append([i,j,np.mean(lst_spe_stratified),np.mean(lst_sens_stratified),np.mean(lst_bala_accu_stratified),
                    np.mean(lst_pre),np.mean(lst_npv),np.mean(lst_exp_cost)])
lis_sv2 = pd.DataFrame(lis)
df_sv2 = lis_sv2.sort_values(by=[4], ascending=False)
print(df_sv2.head(1))

In [ ]:
## StratifiedKFold cross validation ##

#balance = {0:1,1:1000}
c = [50, 10, 1.0, 0.1, 0.01]
ker = ['poly', 'rbf', 'sigmoid']
lis = []
#StratifiedKFold cross validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=10)
#main loop for tuning
for i in c:
    for j in ker:
        lst_bala_accu_stratified = []
        lst_spe_stratified = []
        lst_sens_stratified = []
        lst_exp_cost = []
        lst_pre = []
        lst_npv = []
        for train_index, test_index in skf.split(scaled, y):
            X_train_fold, X_test_fold = scaled[train_index], scaled[test_index]
            y_train_fold, y_test_fold = y[train_index], y[test_index]
            sv = SVC(C = i, kernel = j)
            sv.fit(X_train_fold, y_train_fold)
            sv_prd=sv.predict(X_test_fold)
            tn, fp, fn, tp = confusion_matrix(y_test_fold, sv_prd).ravel()
            ba = 1./2*(1.*tp/(tp+fn)+1.*tn/(tn+fp))
            spe = 1.*tn/(tn+fp)
            sens = 1.*tp/(tp+fn)
            pres = tp/(tp+fp)
            npvs = tn/(tn+fn)
            cos = 565.*fn + 1.*fp
            lst_bala_accu_stratified.append(ba)
            lst_spe_stratified.append(spe)
            lst_sens_stratified.append(sens)
            lst_exp_cost.append(cos)
            lst_pre.append(pres)
            lst_npv.append(npvs)
        #print(np.mean(lst_bala_accu_stratified))
        lis.append([i,j,np.mean(lst_spe_stratified),np.mean(lst_sens_stratified),np.mean(lst_bala_accu_stratified),
                    np.mean(lst_pre),np.mean(lst_npv),np.mean(lst_exp_cost)])
lis_sv0 = pd.DataFrame(lis)
df_sv0 = lis_sv0.sort_values(by=[4], ascending=False)
print(df_sv0.head(1))

In [ ]:
## StratifiedKFold cross validation ##

balance = {0:1,1:1000}
m_dep = [2, 4, 6, 8, 10, 50, 100]
lis = []
#StratifiedKFold cross validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=10)
for i in m_dep:
    lst_bala_accu_stratified = []
    lst_spe_stratified = []
    lst_sens_stratified = []
    lst_exp_cost = []
    lst_pre = []
    lst_npv = []
    for train_index, test_index in skf.split(scaled, y):
            X_train_fold, X_test_fold = scaled[train_index], scaled[test_index]
            y_train_fold, y_test_fold = y[train_index], y[test_index]
            DT = DecisionTreeClassifier(max_depth=i, class_weight=balance)
            DT.fit(X_train_fold, y_train_fold)
            DT_prd=DT.predict(X_test_fold)
            tn, fp, fn, tp = confusion_matrix(y_test_fold, DT_prd).ravel()
            ba = 1./2*(1.*tp/(tp+fn)+1.*tn/(tn+fp))
            spe = 1.*tn/(tn+fp)
            sens = 1.*tp/(tp+fn)
            pres = tp/(tp+fp)
            npvs = tn/(tn+fn)
            cos = 565.*fn + 1.*fp
            lst_bala_accu_stratified.append(ba)
            lst_spe_stratified.append(spe)
            lst_sens_stratified.append(sens)
            lst_exp_cost.append(cos)
            lst_pre.append(pres)
            lst_npv.append(npvs)
    #print(np.mean(lst_bala_accu_stratified))
    lis.append([i,np.mean(lst_spe_stratified),np.mean(lst_sens_stratified),np.mean(lst_bala_accu_stratified),
                    np.mean(lst_pre),np.mean(lst_npv),np.mean(lst_exp_cost)])
lis_dt10 = pd.DataFrame(lis)
df_dt10 = lis_dt10.sort_values(by=[3], ascending=False)
print(df_dt10.head(1))

In [ ]:
## StratifiedKFold cross validation ##

balance = {0:1,1:500}
m_dep = [2, 4, 6, 8, 10, 50, 100]
lis = []
#StratifiedKFold cross validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=10)
for i in m_dep:
    lst_bala_accu_stratified = []
    lst_spe_stratified = []
    lst_sens_stratified = []
    lst_exp_cost = []
    lst_pre = []
    lst_npv = []
    for train_index, test_index in skf.split(scaled, y):
            X_train_fold, X_test_fold = scaled[train_index], scaled[test_index]
            y_train_fold, y_test_fold = y[train_index], y[test_index]
            DT = DecisionTreeClassifier(max_depth=i, class_weight=balance)
            DT.fit(X_train_fold, y_train_fold)
            DT_prd=DT.predict(X_test_fold)
            tn, fp, fn, tp = confusion_matrix(y_test_fold, DT_prd).ravel()
            ba = 1./2*(1.*tp/(tp+fn)+1.*tn/(tn+fp))
            spe = 1.*tn/(tn+fp)
            sens = 1.*tp/(tp+fn)
            pres = tp/(tp+fp)
            npvs = tn/(tn+fn)
            cos = 565.*fn + 1.*fp
            lst_bala_accu_stratified.append(ba)
            lst_spe_stratified.append(spe)
            lst_sens_stratified.append(sens)
            lst_exp_cost.append(cos)
            lst_pre.append(pres)
            lst_npv.append(npvs)
    #print(np.mean(lst_bala_accu_stratified))
    lis.append([i,np.mean(lst_spe_stratified),np.mean(lst_sens_stratified),np.mean(lst_bala_accu_stratified),
                    np.mean(lst_pre),np.mean(lst_npv),np.mean(lst_exp_cost)])
lis_dt5 = pd.DataFrame(lis)
df_dt5 = lis_dt5.sort_values(by=[3], ascending=False)
print(df_dt5.head(1))

In [ ]:
## StratifiedKFold cross validation ##

balance = {0:1,1:200}
m_dep = [2, 4, 6, 8, 10, 50, 100]
lis = []
#StratifiedKFold cross validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=10)
for i in m_dep:
    lst_bala_accu_stratified = []
    lst_spe_stratified = []
    lst_sens_stratified = []
    lst_exp_cost = []
    lst_pre = []
    lst_npv = []
    for train_index, test_index in skf.split(scaled, y):
            X_train_fold, X_test_fold = scaled[train_index], scaled[test_index]
            y_train_fold, y_test_fold = y[train_index], y[test_index]
            DT = DecisionTreeClassifier(max_depth=i, class_weight=balance)
            DT.fit(X_train_fold, y_train_fold)
            DT_prd=DT.predict(X_test_fold)
            tn, fp, fn, tp = confusion_matrix(y_test_fold, DT_prd).ravel()
            ba = 1./2*(1.*tp/(tp+fn)+1.*tn/(tn+fp))
            spe = 1.*tn/(tn+fp)
            sens = 1.*tp/(tp+fn)
            pres = tp/(tp+fp)
            npvs = tn/(tn+fn)
            cos = 565.*fn + 1.*fp
            lst_bala_accu_stratified.append(ba)
            lst_spe_stratified.append(spe)
            lst_sens_stratified.append(sens)
            lst_exp_cost.append(cos)
            lst_pre.append(pres)
            lst_npv.append(npvs)
    #print(np.mean(lst_bala_accu_stratified))
    lis.append([i,np.mean(lst_spe_stratified),np.mean(lst_sens_stratified),np.mean(lst_bala_accu_stratified),
                    np.mean(lst_pre),np.mean(lst_npv),np.mean(lst_exp_cost)])
lis_dt2 = pd.DataFrame(lis)
df_dt2 = lis_dt2.sort_values(by=[3], ascending=False)
print(df_dt2.head(1))

In [ ]:
## StratifiedKFold cross validation ##

balance = {0:1,1:300}
m_dep = [2, 4, 6, 8, 10, 50, 100]
lis = []
#StratifiedKFold cross validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=10)
for i in m_dep:
    lst_bala_accu_stratified = []
    lst_spe_stratified = []
    lst_sens_stratified = []
    lst_exp_cost = []
    lst_pre = []
    lst_npv = []
    for train_index, test_index in skf.split(scaled, y):
            X_train_fold, X_test_fold = scaled[train_index], scaled[test_index]
            y_train_fold, y_test_fold = y[train_index], y[test_index]
            DT = DecisionTreeClassifier(max_depth=i, class_weight=balance)
            DT.fit(X_train_fold, y_train_fold)
            DT_prd=DT.predict(X_test_fold)
            tn, fp, fn, tp = confusion_matrix(y_test_fold, DT_prd).ravel()
            ba = 1./2*(1.*tp/(tp+fn)+1.*tn/(tn+fp))
            spe = 1.*tn/(tn+fp)
            sens = 1.*tp/(tp+fn)
            pres = tp/(tp+fp)
            npvs = tn/(tn+fn)
            cos = 565.*fn + 1.*fp
            lst_bala_accu_stratified.append(ba)
            lst_spe_stratified.append(spe)
            lst_sens_stratified.append(sens)
            lst_exp_cost.append(cos)
            lst_pre.append(pres)
            lst_npv.append(npvs)
    #print(np.mean(lst_bala_accu_stratified))
    lis.append([i,np.mean(lst_spe_stratified),np.mean(lst_sens_stratified),np.mean(lst_bala_accu_stratified),
                    np.mean(lst_pre),np.mean(lst_npv),np.mean(lst_exp_cost)])
lis_dt3 = pd.DataFrame(lis)
df_dt3 = lis_dt3.sort_values(by=[3], ascending=False)
print(df_dt3.head(1))

In [ ]:
## StratifiedKFold cross validation ##

#balance = {0:1,1:300}
m_dep = [2, 4, 6, 8, 10, 50, 100]
lis = []
#StratifiedKFold cross validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=10)
for i in m_dep:
    lst_bala_accu_stratified = []
    lst_spe_stratified = []
    lst_sens_stratified = []
    lst_exp_cost = []
    lst_pre = []
    lst_npv = []
    for train_index, test_index in skf.split(scaled, y):
            X_train_fold, X_test_fold = scaled[train_index], scaled[test_index]
            y_train_fold, y_test_fold = y[train_index], y[test_index]
            DT = DecisionTreeClassifier(max_depth=i)
            DT.fit(X_train_fold, y_train_fold)
            DT_prd=DT.predict(X_test_fold)
            tn, fp, fn, tp = confusion_matrix(y_test_fold, DT_prd).ravel()
            ba = 1./2*(1.*tp/(tp+fn)+1.*tn/(tn+fp))
            spe = 1.*tn/(tn+fp)
            sens = 1.*tp/(tp+fn)
            pres = tp/(tp+fp)
            npvs = tn/(tn+fn)
            cos = 565.*fn + 1.*fp
            lst_bala_accu_stratified.append(ba)
            lst_spe_stratified.append(spe)
            lst_sens_stratified.append(sens)
            lst_exp_cost.append(cos)
            lst_pre.append(pres)
            lst_npv.append(npvs)
    #print(np.mean(lst_bala_accu_stratified))
    lis.append([i,np.mean(lst_spe_stratified),np.mean(lst_sens_stratified),np.mean(lst_bala_accu_stratified),
                    np.mean(lst_pre),np.mean(lst_npv),np.mean(lst_exp_cost)])
lis_dt0 = pd.DataFrame(lis)
df_dt0 = lis_dt0.sort_values(by=[3], ascending=False)
print(df_dt0.head(1))

In [ ]:
## StratifiedKFold cross validation ##

balance = {0:1,1:200}
c = [0.01, 0.1, 1, 10, 100]
sol = ['liblinear','saga']
lis = []
#StratifiedKFold cross validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=10)
#main loop for tuning
for i in c:
    for j in sol:
        lst_bala_accu_stratified = []
        lst_spe_stratified = []
        lst_sens_stratified = []
        lst_exp_cost = []
        lst_pre = []
        lst_npv = []
        for train_index, test_index in skf.split(scaled, y):
            X_train_fold, X_test_fold = scaled[train_index], scaled[test_index]
            y_train_fold, y_test_fold = y[train_index], y[test_index]
            LogReg1 = LogisticRegression(C=i, penalty='l1', solver=j, class_weight=balance)
            LogReg1.fit(X_train_fold, y_train_fold)
            LogReg1_prd=LogReg1.predict(X_test_fold)
            tn, fp, fn, tp = confusion_matrix(y_test_fold, LogReg1_prd).ravel()
            ba = 1./2*(1.*tp/(tp+fn)+1.*tn/(tn+fp))
            spe = 1.*tn/(tn+fp)
            sens = 1.*tp/(tp+fn)
            pres = tp/(tp+fp)
            npvs = tn/(tn+fn)
            cos = 565.*fn + 1.*fp
            lst_bala_accu_stratified.append(ba)
            lst_spe_stratified.append(spe)
            lst_sens_stratified.append(sens)
            lst_exp_cost.append(cos)
            lst_pre.append(pres)
            lst_npv.append(npvs)
        #print(np.mean(lst_bala_accu_stratified))
        lis.append([i,j,np.mean(lst_spe_stratified),np.mean(lst_sens_stratified),np.mean(lst_bala_accu_stratified),
                    np.mean(lst_pre),np.mean(lst_npv),np.mean(lst_exp_cost)])
lis_lr2 = pd.DataFrame(lis)
df_lr2 = lis_lr2.sort_values(by=[4], ascending=False)
print(df_lr2.head(1))

In [ ]:
## StratifiedKFold cross validation ##

balance = {0:1,1:300}
c = [0.01, 0.1, 1, 10, 100]
sol = ['liblinear','saga']
lis = []
#StratifiedKFold cross validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=10)
#main loop for tuning
for i in c:
    for j in sol:
        lst_bala_accu_stratified = []
        lst_spe_stratified = []
        lst_sens_stratified = []
        lst_exp_cost = []
        lst_pre = []
        lst_npv = []
        for train_index, test_index in skf.split(scaled, y):
            X_train_fold, X_test_fold = scaled[train_index], scaled[test_index]
            y_train_fold, y_test_fold = y[train_index], y[test_index]
            LogReg1 = LogisticRegression(C=i, penalty='l1', solver=j, class_weight=balance)
            LogReg1.fit(X_train_fold, y_train_fold)
            LogReg1_prd=LogReg1.predict(X_test_fold)
            tn, fp, fn, tp = confusion_matrix(y_test_fold, LogReg1_prd).ravel()
            ba = 1./2*(1.*tp/(tp+fn)+1.*tn/(tn+fp))
            spe = 1.*tn/(tn+fp)
            sens = 1.*tp/(tp+fn)
            pres = tp/(tp+fp)
            npvs = tn/(tn+fn)
            cos = 565.*fn + 1.*fp
            lst_bala_accu_stratified.append(ba)
            lst_spe_stratified.append(spe)
            lst_sens_stratified.append(sens)
            lst_exp_cost.append(cos)
            lst_pre.append(pres)
            lst_npv.append(npvs)
        #print(np.mean(lst_bala_accu_stratified))
        lis.append([i,j,np.mean(lst_spe_stratified),np.mean(lst_sens_stratified),np.mean(lst_bala_accu_stratified),
                    np.mean(lst_pre),np.mean(lst_npv),np.mean(lst_exp_cost)])
lis_lr31 = pd.DataFrame(lis)
df_lr31 = lis_lr31.sort_values(by=[4], ascending=False)
print(df_lr31.head(1))

In [ ]:
## StratifiedKFold cross validation ##

balance = {0:1,1:500}
c = [0.01, 0.1, 1, 10, 100]
sol = ['liblinear','saga']
lis = []
#StratifiedKFold cross validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=10)
#main loop for tuning
for i in c:
    for j in sol:
        lst_bala_accu_stratified = []
        lst_spe_stratified = []
        lst_sens_stratified = []
        lst_exp_cost = []
        lst_pre = []
        lst_npv = []
        for train_index, test_index in skf.split(scaled, y):
            X_train_fold, X_test_fold = scaled[train_index], scaled[test_index]
            y_train_fold, y_test_fold = y[train_index], y[test_index]
            LogReg1 = LogisticRegression(C=i, penalty='l1', solver=j, class_weight=balance)
            LogReg1.fit(X_train_fold, y_train_fold)
            LogReg1_prd=LogReg1.predict(X_test_fold)
            tn, fp, fn, tp = confusion_matrix(y_test_fold, LogReg1_prd).ravel()
            ba = 1./2*(1.*tp/(tp+fn)+1.*tn/(tn+fp))
            spe = 1.*tn/(tn+fp)
            sens = 1.*tp/(tp+fn)
            pres = tp/(tp+fp)
            npvs = tn/(tn+fn)
            cos = 565.*fn + 1.*fp
            lst_bala_accu_stratified.append(ba)
            lst_spe_stratified.append(spe)
            lst_sens_stratified.append(sens)
            lst_exp_cost.append(cos)
            lst_pre.append(pres)
            lst_npv.append(npvs)
        #print(np.mean(lst_bala_accu_stratified))
        lis.append([i,j,np.mean(lst_spe_stratified),np.mean(lst_sens_stratified),np.mean(lst_bala_accu_stratified),
                    np.mean(lst_pre),np.mean(lst_npv),np.mean(lst_exp_cost)])
lis_lr5 = pd.DataFrame(lis)
df_lr5 = lis_lr5.sort_values(by=[4], ascending=False)
print(df_lr5.head(1))

In [ ]:
## StratifiedKFold cross validation ##

balance = {0:1,1:1000}
c = [0.01, 0.1, 1, 10, 100]
sol = ['liblinear','saga']
lis = []
#StratifiedKFold cross validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=10)
#main loop for tuning
for i in c:
    for j in sol:
        lst_bala_accu_stratified = []
        lst_spe_stratified = []
        lst_sens_stratified = []
        lst_exp_cost = []
        lst_pre = []
        lst_npv = []
        for train_index, test_index in skf.split(scaled, y):
            X_train_fold, X_test_fold = scaled[train_index], scaled[test_index]
            y_train_fold, y_test_fold = y[train_index], y[test_index]
            LogReg1 = LogisticRegression(C=i, penalty='l1', solver=j, class_weight=balance)
            LogReg1.fit(X_train_fold, y_train_fold)
            LogReg1_prd=LogReg1.predict(X_test_fold)
            tn, fp, fn, tp = confusion_matrix(y_test_fold, LogReg1_prd).ravel()
            ba = 1./2*(1.*tp/(tp+fn)+1.*tn/(tn+fp))
            spe = 1.*tn/(tn+fp)
            sens = 1.*tp/(tp+fn)
            pres = tp/(tp+fp)
            npvs = tn/(tn+fn)
            cos = 565.*fn + 1.*fp
            lst_bala_accu_stratified.append(ba)
            lst_spe_stratified.append(spe)
            lst_sens_stratified.append(sens)
            lst_exp_cost.append(cos)
            lst_pre.append(pres)
            lst_npv.append(npvs)
        #print(np.mean(lst_bala_accu_stratified))
        lis.append([i,j,np.mean(lst_spe_stratified),np.mean(lst_sens_stratified),np.mean(lst_bala_accu_stratified),
                    np.mean(lst_pre),np.mean(lst_npv),np.mean(lst_exp_cost)])
lis_lr10 = pd.DataFrame(lis)
df_lr10 = lis_lr10.sort_values(by=[4], ascending=False)
print(df_lr10.head(1))

In [ ]:
## StratifiedKFold cross validation ##

#balance = {0:1,1:200}
c = [0.01, 0.1, 1, 10, 100]
sol = ['liblinear','saga']
lis = []
#StratifiedKFold cross validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=10)
#main loop for tuning
for i in c:
    for j in sol:
        lst_bala_accu_stratified = []
        lst_spe_stratified = []
        lst_sens_stratified = []
        lst_exp_cost = []
        lst_pre = []
        lst_npv = []
        for train_index, test_index in skf.split(scaled, y):
            X_train_fold, X_test_fold = scaled[train_index], scaled[test_index]
            y_train_fold, y_test_fold = y[train_index], y[test_index]
            LogReg1 = LogisticRegression(C=i, penalty='l1', solver=j)
            LogReg1.fit(X_train_fold, y_train_fold)
            LogReg1_prd=LogReg1.predict(X_test_fold)
            tn, fp, fn, tp = confusion_matrix(y_test_fold, LogReg1_prd).ravel()
            ba = 1./2*(1.*tp/(tp+fn)+1.*tn/(tn+fp))
            spe = 1.*tn/(tn+fp)
            sens = 1.*tp/(tp+fn)
            pres = tp/(tp+fp)
            npvs = tn/(tn+fn)
            cos = 565.*fn + 1.*fp
            lst_bala_accu_stratified.append(ba)
            lst_spe_stratified.append(spe)
            lst_sens_stratified.append(sens)
            lst_exp_cost.append(cos)
            lst_pre.append(pres)
            lst_npv.append(npvs)
        #print(np.mean(lst_bala_accu_stratified))
        lis.append([i,j,np.mean(lst_spe_stratified),np.mean(lst_sens_stratified),np.mean(lst_bala_accu_stratified),
                    np.mean(lst_pre),np.mean(lst_npv),np.mean(lst_exp_cost)])
lis_lr0 = pd.DataFrame(lis)
df_lr0 = lis_lr0.sort_values(by=[4], ascending=False)
print(df_lr0.head(1))

In [ ]:
## StratifiedKFold cross validation ##

balance = {0:1,1:200}
c = [0.01, 0.1, 1, 10, 100]
sol = ['newton-cg', 'lbfgs', 'liblinear']
lis = []
#StratifiedKFold cross validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=10)
#main loop for tuning
for i in c:
    for j in sol:
        lst_bala_accu_stratified = []
        lst_spe_stratified = []
        lst_sens_stratified = []
        for train_index, test_index in skf.split(X, y):
            X_train_fold, X_test_fold = X.loc[train_index], X.loc[test_index]
            y_train_fold, y_test_fold = y[train_index], y[test_index]
            LogReg = LogisticRegression(C=i, penalty='l2', solver=j, class_weight=balance)
            LogReg.fit(X_train_fold, y_train_fold)
            LogReg_prd=LogReg.predict(X_test_fold)
            tn, fp, fn, tp = confusion_matrix(y_test_fold, LogReg_prd).ravel()
            ba = 1./2*(1.*tp/(tp+fn)+1.*tn/(tn+fp))
            spe = 1.*tn/(tn+fp)
            sens = 1.*tp/(tp+fn)
            lst_bala_accu_stratified.append(ba)
            lst_spe_stratified.append(spe)
            lst_sens_stratified.append(sens)
        #print(np.mean(lst_bala_accu_stratified))
        lis.append([i,j,np.mean(lst_spe_stratified),np.mean(lst_sens_stratified),np.mean(lst_bala_accu_stratified)])
lis_lr = pd.DataFrame(lis)
df_lr = lis_lr.sort_values(by=[4], ascending=False)
print(df_lr.head(1))

In [ ]:
## StratifiedKFold cross validation ##

balance = {0:1,1:300}
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=10)
lst_bala_accu_stratified = []
lst_spe_stratified = []
lst_sens_stratified = []
for train_index, test_index in skf.split(X, y):
    X_train_fold, X_test_fold = X.loc[train_index], X.loc[test_index]
    y_train_fold, y_test_fold = y[train_index], y[test_index]
    LogReg = LogisticRegression(C=0.1, penalty='l2', solver='liblinear', class_weight=balance)
    LogReg.fit(X_train_fold, y_train_fold)
    LogReg_prd=LogReg.predict(X_test_fold)
    tn, fp, fn, tp = confusion_matrix(y_test_fold, LogReg_prd).ravel()
    ba = 1./2*(1.*tp/(tp+fn)+1.*tn/(tn+fp))
    spe = 1.*tn/(tn+fp)
    sens = 1.*tp/(tp+fn)
    lst_bala_accu_stratified.append(ba)
    lst_spe_stratified.append(spe)
    lst_sens_stratified.append(sens)
print(lst_bala_accu_stratified)

In [ ]:
## StratifiedKFold cross validation ##

balance = {0:1,1:300}
c = [0.01, 0.1, 1, 10, 100]
sol = ['newton-cg', 'lbfgs', 'liblinear']
lis = []
#StratifiedKFold cross validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=10)
#main loop for tuning
for i in c:
    for j in sol:
        lst_bala_accu_stratified = []
        lst_spe_stratified = []
        lst_sens_stratified = []
        for train_index, test_index in skf.split(X_nobp, y):
            X_train_fold, X_test_fold = X_nobp.loc[train_index], X_nobp.loc[test_index]
            y_train_fold, y_test_fold = y[train_index], y[test_index]
            LogReg1 = LogisticRegression(C=i, penalty='l2', solver=j, class_weight=balance)
            LogReg1.fit(X_train_fold, y_train_fold)
            LogReg1_prd=LogReg1.predict(X_test_fold)
            tn, fp, fn, tp = confusion_matrix(y_test_fold, LogReg1_prd).ravel()
            ba = 1./2*(1.*tp/(tp+fn)+1.*tn/(tn+fp))
            spe = 1.*tn/(tn+fp)
            sens = 1.*tp/(tp+fn)
            lst_bala_accu_stratified.append(ba)
            lst_spe_stratified.append(spe)
            lst_sens_stratified.append(sens)
        #print(np.mean(lst_bala_accu_stratified))
        lis.append([i,j,np.mean(lst_spe_stratified),np.mean(lst_sens_stratified),np.mean(lst_bala_accu_stratified)])
lis_lr = pd.DataFrame(lis)
df_lr = lis_lr.sort_values(by=[4], ascending=False)
print(df_lr.head(1))

In [ ]:
## StratifiedKFold cross validation ##

balance = {0:1,1:300}
c = [0.01, 0.1, 1, 10, 100]
sol = ['newton-cg', 'lbfgs', 'liblinear']
lis = []
#StratifiedKFold cross validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=10)
#main loop for tuning
for i in c:
    for j in sol:
        lst_bala_accu_stratified = []
        lst_spe_stratified = []
        lst_sens_stratified = []
        for train_index, test_index in skf.split(X, y):
            X_train_fold, X_test_fold = X.loc[train_index], X.loc[test_index]
            y_train_fold, y_test_fold = y[train_index], y[test_index]
            LogReg1 = LogisticRegression(C=i, penalty='l2', solver=j, class_weight=balance)
            LogReg1.fit(X_train_fold, y_train_fold)
            LogReg1_prd=LogReg1.predict(X_test_fold)
            tn, fp, fn, tp = confusion_matrix(y_test_fold, LogReg1_prd).ravel()
            ba = 1./2*(1.*tp/(tp+fn)+1.*tn/(tn+fp))
            spe = 1.*tn/(tn+fp)
            sens = 1.*tp/(tp+fn)
            lst_bala_accu_stratified.append(ba)
            lst_spe_stratified.append(spe)
            lst_sens_stratified.append(sens)
        #print(np.mean(lst_bala_accu_stratified))
        lis.append([i,j,np.mean(lst_spe_stratified),np.mean(lst_sens_stratified),np.mean(lst_bala_accu_stratified)])
lis_lr = pd.DataFrame(lis)
df_lr = lis_lr.sort_values(by=[4], ascending=False)
print(df_lr.head(1))

In [ ]:
## StratifiedKFold cross validation ##

balance = {0:1,1:500}
c = [0.01, 0.1, 1, 10, 100]
sol = ['newton-cg', 'lbfgs', 'liblinear']
lis = []
#StratifiedKFold cross validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=10)
#main loop for tuning
for i in c:
    for j in sol:
        lst_bala_accu_stratified = []
        lst_spe_stratified = []
        lst_sens_stratified = []
        for train_index, test_index in skf.split(X, y):
            X_train_fold, X_test_fold = X.loc[train_index], X.loc[test_index]
            y_train_fold, y_test_fold = y[train_index], y[test_index]
            LogReg1 = LogisticRegression(C=i, penalty='l2', solver=j, class_weight=balance)
            LogReg1.fit(X_train_fold, y_train_fold)
            LogReg1_prd=LogReg1.predict(X_test_fold)
            tn, fp, fn, tp = confusion_matrix(y_test_fold, LogReg1_prd).ravel()
            ba = 1./2*(1.*tp/(tp+fn)+1.*tn/(tn+fp))
            spe = 1.*tn/(tn+fp)
            sens = 1.*tp/(tp+fn)
            lst_bala_accu_stratified.append(ba)
            lst_spe_stratified.append(spe)
            lst_sens_stratified.append(sens)
        #print(np.mean(lst_bala_accu_stratified))
        lis.append([i,j,np.mean(lst_spe_stratified),np.mean(lst_sens_stratified),np.mean(lst_bala_accu_stratified)])
lis_lr = pd.DataFrame(lis)
df_lr = lis_lr.sort_values(by=[4], ascending=False)
print(df_lr.head(1))

In [ ]:
## StratifiedKFold cross validation ##

balance = {0:1,1:1000}
c = [0.01, 0.1, 1, 10, 100]
sol = ['newton-cg', 'lbfgs', 'liblinear']
lis = []
#StratifiedKFold cross validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=10)
#main loop for tuning
for i in c:
    for j in sol:
        lst_bala_accu_stratified = []
        lst_spe_stratified = []
        lst_sens_stratified = []
        for train_index, test_index in skf.split(X, y):
            X_train_fold, X_test_fold = X.loc[train_index], X.loc[test_index]
            y_train_fold, y_test_fold = y[train_index], y[test_index]
            LogReg1 = LogisticRegression(C=i, penalty='l2', solver=j, class_weight=balance)
            LogReg1.fit(X_train_fold, y_train_fold)
            LogReg1_prd=LogReg1.predict(X_test_fold)
            tn, fp, fn, tp = confusion_matrix(y_test_fold, LogReg1_prd).ravel()
            ba = 1./2*(1.*tp/(tp+fn)+1.*tn/(tn+fp))
            spe = 1.*tn/(tn+fp)
            sens = 1.*tp/(tp+fn)
            lst_bala_accu_stratified.append(ba)
            lst_spe_stratified.append(spe)
            lst_sens_stratified.append(sens)
        #print(np.mean(lst_bala_accu_stratified))
        lis.append([i,j,np.mean(lst_spe_stratified),np.mean(lst_sens_stratified),np.mean(lst_bala_accu_stratified)])
lis_lr = pd.DataFrame(lis)
df_lr = lis_lr.sort_values(by=[4], ascending=False)
print(df_lr.head(1))

In [ ]:
## StratifiedKFold cross validation ##

#balance = {0:1,1:1000}
c = [0.01, 0.1, 1, 10, 100]
sol = ['newton-cg', 'lbfgs', 'liblinear']
lis = []
#StratifiedKFold cross validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=10)
#main loop for tuning
for i in c:
    for j in sol:
        lst_bala_accu_stratified = []
        lst_spe_stratified = []
        lst_sens_stratified = []
        for train_index, test_index in skf.split(X, y):
            X_train_fold, X_test_fold = X.loc[train_index], X.loc[test_index]
            y_train_fold, y_test_fold = y[train_index], y[test_index]
            LogReg1 = LogisticRegression(C=i, penalty='l2', solver=j)
            LogReg1.fit(X_train_fold, y_train_fold)
            LogReg1_prd=LogReg1.predict(X_test_fold)
            tn, fp, fn, tp = confusion_matrix(y_test_fold, LogReg1_prd).ravel()
            ba = 1./2*(1.*tp/(tp+fn)+1.*tn/(tn+fp))
            spe = 1.*tn/(tn+fp)
            sens = 1.*tp/(tp+fn)
            lst_bala_accu_stratified.append(ba)
            lst_spe_stratified.append(spe)
            lst_sens_stratified.append(sens)
        #print(np.mean(lst_bala_accu_stratified))
        lis.append([i,j,np.mean(lst_spe_stratified),np.mean(lst_sens_stratified),np.mean(lst_bala_accu_stratified)])
lis_lr = pd.DataFrame(lis)
df_lr = lis_lr.sort_values(by=[4], ascending=False)
print(df_lr.head(1))

# Performance of predictive models

In [ ]:
## ROC curves ##


import sklearn.metrics as metrics
# calculate the fpr and tpr for all thresholds of the classification
balance = {0:1,1:500}
classifier = RandomForestClassifier(max_depth = 6, n_estimators = 1000, class_weight=balance)
classifier.fit(X, y)
probs = classifier.predict_proba(X)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y, preds)
roc_auc = metrics.auc(fpr, tpr)
classifier.fit(X_nobp, y)
probs_n = classifier.predict_proba(X_nobp)
preds_n = probs_n[:,1]
fpr_n, tpr_n, threshold = metrics.roc_curve(y, preds_n)
roc_auc_n = metrics.auc(fpr_n, tpr_n)
# method I: plt
import matplotlib.pyplot as plt
plt.title('ROC curve with and without BP features\n(Positive label "Readmission")')
plt.plot(fpr, tpr, 'b', label = 'AUC with BP = %0.2f' % roc_auc)
plt.plot(fpr_n, tpr_n, 'orange', label = 'AUC without BP = %0.2f' % roc_auc_n)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'k--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
#plt.show()
plt.savefig('ROC_compare.png')

In [29]:
from scipy import interp
import matplotlib.patches as patches
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
## More ROC curves ##

balance = {0:1,1:500}
#cv = StratifiedKFold(n_splits=5)
classifier = RandomForestClassifier(max_depth = 6, n_estimators = 1000, class_weight=balance, random_state=10)
#clf = RandomForestClassifier(random_state=random_state)
cv = StratifiedKFold(n_splits=10,shuffle=False)
# plot arrows
fig1 = plt.figure(figsize=[6,6])
ax1 = fig1.add_subplot(111,aspect = 'equal')
#ax1.add_patch(
#    patches.Arrow(0.45,0.5,-0.25,0.25,width=0.3,color='green',alpha = 0.5)
#    )
#ax1.add_patch(
#    patches.Arrow(0.5,0.45,0.25,-0.25,width=0.3,color='red',alpha = 0.5)
#    )

tprs = []
aucs = []
mean_fpr = np.linspace(0,1,100)
#i = 1
for train,test in cv.split(X,y):
    prediction = classifier.fit(X.iloc[train],y.iloc[train]).predict_proba(X.iloc[test])
    fpr, tpr, t = roc_curve(y[test], prediction[:, 1])
    tprs.append(interp(mean_fpr, fpr, tpr))
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
    #plt.plot(fpr, tpr, lw=2, alpha=0.3, label='ROC fold %d (AUC = %0.2f)' % (i, roc_auc))
    #i= i+1

plt.plot([0,1],[0,1],linestyle = '--',lw = 2,color = 'black')
mean_tpr = np.mean(tprs, axis=0)
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
plt.plot(mean_fpr, mean_tpr, color='blue',
         label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),lw=2, alpha=1)

tprs_n = []
aucs_n = []
mean_fpr_n = np.linspace(0,1,100)
#i = 1
for train,test in cv.split(X_nobp,y):
    prediction_n = classifier.fit(X_nobp.iloc[train],y.iloc[train]).predict_proba(X_nobp.iloc[test])
    fpr, tpr, t = roc_curve(y[test], prediction_n[:, 1])
    tprs_n.append(interp(mean_fpr_n, fpr, tpr))
    roc_auc = auc(fpr, tpr)
    aucs_n.append(roc_auc)
    #plt.plot(fpr, tpr, lw=2, alpha=0.3, label='ROC fold %d (AUC = %0.2f)' % (i, roc_auc))
    #i= i+1

#plt.plot([0,1],[0,1],linestyle = '--',lw = 2,color = 'black')
mean_tpr_n = np.mean(tprs_n, axis=0)
mean_auc_n = auc(mean_fpr_n, mean_tpr_n)
std_auc_n = np.std(aucs_n)
plt.plot(mean_fpr_n, mean_tpr_n, color='orange',
         label=r'Mean ROC WITHOUT BP (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc_n, std_auc_n),lw=2, alpha=1)

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve with and without BP features')
plt.legend(loc="lower right")
#plt.text(0.32,0.7,'More accurate area',fontsize = 12)
#plt.text(0.63,0.4,'Less accurate area',fontsize = 12)
plt.savefig('final_Mean_ROC.png')
#plt.show()

In [ ]:
## More ROC curves ##

import matplotlib.pyplot as plt

from sklearn.metrics import auc
from sklearn.metrics import RocCurveDisplay
from sklearn.model_selection import StratifiedKFold

balance = {0:1,1:300}
cv = StratifiedKFold(n_splits=5)
classifier = LogisticRegression(C=0.1, penalty='l2', solver='liblinear', class_weight=balance)

tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)

fig, ax = plt.subplots(figsize=(6, 6))
for fold, (train, test) in enumerate(cv.split(X, y)):
    classifier.fit(X.loc[train], y[train])
    viz = RocCurveDisplay.from_estimator(
        classifier,
        X.loc[test],
        y[test],
        name=f"ROC fold {fold}",
        alpha=0.3,
        lw=1,
        ax=ax,
    )
    interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)
    aucs.append(viz.roc_auc)
ax.plot([0, 1], [0, 1], "k--", label="chance level (AUC = 0.5)")

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
ax.plot(
    mean_fpr,
    mean_tpr,
    color="b",
    label=r"Mean ROC (AUC = %0.2f $\pm$ %0.2f)" % (mean_auc, std_auc),
    lw=2,
    alpha=0.8,
)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
ax.fill_between(
    mean_fpr,
    tprs_lower,
    tprs_upper,
    color="grey",
    alpha=0.2,
    label=r"$\pm$ 1 std. dev.",
)

ax.set(
    xlim=[-0.05, 1.05],
    ylim=[-0.05, 1.05],
    xlabel="False Positive Rate",
    ylabel="True Positive Rate",
    title=f"Mean ROC curve with BP features\n(Positive label 'Readmission')",
)
ax.axis("square")
ax.legend(loc="lower right")
#plt.show()
plt.savefig('Mean_ROC_bp.png')

In [ ]:
## More ROC curves ##

import matplotlib.pyplot as plt

from sklearn.metrics import auc
from sklearn.metrics import RocCurveDisplay
from sklearn.model_selection import StratifiedKFold

balance = {0:1,1:300}
cv = StratifiedKFold(n_splits=5)
classifier = LogisticRegression(C=0.1, penalty='l2', solver='liblinear', class_weight=balance)

tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)

fig, ax = plt.subplots(figsize=(6, 6))
for fold, (train, test) in enumerate(cv.split(X_nobp, y)):
    classifier.fit(X_nobp.loc[train], y[train])
    viz = RocCurveDisplay.from_estimator(
        classifier,
        X_nobp.loc[test],
        y[test],
        name=f"ROC fold {fold}",
        alpha=0.3,
        lw=1,
        ax=ax,
    )
    interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)
    aucs.append(viz.roc_auc)
ax.plot([0, 1], [0, 1], "k--", label="chance level (AUC = 0.5)")

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
ax.plot(
    mean_fpr,
    mean_tpr,
    color="b",
    label=r"Mean ROC (AUC = %0.2f $\pm$ %0.2f)" % (mean_auc, std_auc),
    lw=2,
    alpha=0.8,
)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
ax.fill_between(
    mean_fpr,
    tprs_lower,
    tprs_upper,
    color="grey",
    alpha=0.2,
    label=r"$\pm$ 1 std. dev.",
)

ax.set(
    xlim=[-0.05, 1.05],
    ylim=[-0.05, 1.05],
    xlabel="False Positive Rate",
    ylabel="True Positive Rate",
    title=f"Mean ROC curve without BP features\n(Positive label 'Readmission')",
)
ax.axis("square")
ax.legend(loc="lower right")
#plt.show()
plt.savefig('Mean_ROC_wo_bp.png')

In [ ]:
#for Random Forest
y_predRF_c5 = RF_c5.predict_proba(X_test).T[1]
fprRf5, tprRf5, threshRf5 = roc_curve(y_test, y_predRF_c5)
roc_auc_rf5= roc_auc_score(y_test, y_predRF_c5)

y_predRF_c2 = RF_c2.predict_proba(X_test).T[1]
fprRf2, tprRf2, threshRf2 = roc_curve(y_test, y_predRF_c2)
roc_auc_rf2= roc_auc_score(y_test, y_predRF_c2)

y_predRF_c3 = RF_c3.predict_proba(X_test).T[1]
fprRf3, tprRf3, threshRf3 = roc_curve(y_test, y_predRF_c3)
roc_auc_rf3= roc_auc_score(y_test, y_predRF_c3)

y_predRF_c10 = RF_c10.predict_proba(X_test).T[1]
fprRf10, tprRf10, threshRf10 = roc_curve(y_test, y_predRF_c10)
roc_auc_rf10= roc_auc_score(y_test, y_predRF_c10)

fig = plt.figure(figsize=(12,8))
plt.plot([0, 1], [0, 1], linestyle='--')

plt.plot(fprRf2, tprRf2, label='cost ratio 200 (AUC = %0.3f)' % roc_auc_rf2) 
plt.plot(fprRf3, tprRf3, label='cost ratio 300 (AUC = %0.3f)' % roc_auc_rf3) 
plt.plot(fprRf5, tprRf5, label='cost ratio 500 (AUC = %0.3f)' % roc_auc_rf5) 
plt.plot(fprRf10, tprRf10, label='cost ratio 1000 (AUC = %0.3f)' % roc_auc_rf10) 

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.title('ROC Comparison of Different Models with BP features', fontsize = 15)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right');
plt.savefig('ROC_wo_bp.png')
#plt.show()

In [31]:
## Confusion matrix ##

def display_summary(true,pred):
    tn, fp, fn, tp = confusion_matrix(true,pred).ravel()
    print('confusion matrix')
    print(np.array([[tp,fp],[fn,tn]]))
    print('sensitivity is %f',1.*tp/(tp+fn))
    print('specificity is %f',1.*tn/(tn+fp))
    print('accuracy is %f',1.*(tp+tn)/(tp+tn+fp+fn))
    print('balanced accuracy is %',1./2*(1.*tp/(tp+fn)+1.*tn/(tn+fp)))

In [ ]:
## Leave one out prediction performance ##

from sklearn.model_selection import LeaveOneOut
# Define the balance dictionary
balance = {0: 1, 1: 500}

# Initialize RandomForestClassifier parameters
max_depth = 6
n_estimators = 1000
random_state = 10

# Initialize the RandomForestClassifier
RF_c5 = RandomForestClassifier(max_depth=max_depth, n_estimators=n_estimators, random_state=random_state, class_weight=balance)
loo = LeaveOneOut()
results = np.empty(len(X))


for i, (train_index, test_index) in enumerate(loo.split(X)):
    print(i)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Fit the model on the training data
    RF_c5.fit(X_train, y_train)

    y_probabilities = RF_c5.predict_proba(X_test)
    results[i] = y_probabilities[:, 1][0]  # Access probabilities for class 1 (positive class) in each iteration
    #print(results[i])



In [65]:
## Some key metrics measured on different prediction threshold ##
num_samples = len(X)

# Initialize variables to store results
custom_threshold = [round(i, 2) for i in range(5, 90, 5)]
custom_threshold = [x / 100 for x in custom_threshold]

lst_bala_accu = []
lst_preci = []
lst_exp_cost = []
        
# Compute confusion matrix for each threshold
for i in custom_threshold:
    y_custom_predictions = (results > i).astype(int)
    tn, fp, fn, tp = confusion_matrix(y, y_custom_predictions).ravel()
    ba = 1./2*(1.*tp/(tp+fn)+1.*tn/(tn+fp))
    preci = tp/(tp + fp)
    cos = (565.*fn + 1.*fp)*36
    lst_bala_accu.append(ba)
    lst_preci.append(preci)
    lst_exp_cost.append(cos)

In [ ]:
# Plot the relationship between custom threshold and expected cost
plt.figure(figsize=(8, 6))
plt.plot(custom_threshold, lst_exp_cost)
plt.title('Relationship between Predictive Threshold and Expected Medical Costs')
plt.xlabel('Predictive Threshold')
plt.ylabel('Expected Medical Costs')
#plt.xticks(custom_threshold)
plt.legend()
plt.grid(True)
plt.savefig('predictive_threshold_cost.png')

In [ ]:
# Plot the relationship between custom threshold and accuracy
plt.figure(figsize=(8, 6))
plt.plot(custom_threshold, lst_bala_accu)
plt.title('Relationship between Predictive Threshold and Balanced Accuracy')
plt.xlabel('Predictive Threshold')
plt.ylabel('Balanced Accuracy')
plt.legend()
plt.grid(True)
plt.savefig('predictive_threshold_balanced_accuracy.png')

In [192]:
## Dump the best model for calculator ##

from sklearn.externals import joblib
joblib.dump(RF_c5, '/Users/Troy-T/Downloads/model.joblib')

['/Users/Troy-T/Downloads/model.joblib']